<span style="color:magenta; font-size:32px; font-family:'Times New Roman'">**MecaChess Robotic Arm**</span>

<!-- 

color: cambia el color del texto 
font-size: establece el tamaño de la letra en píxeles.
font-family:'Times New Roman' cambia el tipo de letra a "Times New Roman".
** se utiliza para hacer que el texto del título esté en negrita

-->

<span style="color:blue; font-size:32px; font-family:'Times New Roman'">**Color Extraction Calibration**</span>

<!-- 

color: cambia el color del texto 
font-size: establece el tamaño de la letra en píxeles.
font-family:'Times New Roman' cambia el tipo de letra a "Times New Roman".
** se utiliza para hacer que el texto del título esté en negrita

-->

In [12]:
import cv2
import numpy as np
import os

#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGD                          CALIBRACION DE COLORES                                GDGDGDGDGD    
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD

# Variables globales para almacenar los valores de color seleccionados y las imágenes
selected_colors = []                           # Lista para almacenar los valores seleccionados
hsv_image = None                               # Almacena la imagen en formato HSV
scaled_image = None                            # Almacena la imagen redimensionada

# Variables globales para almacenar hasta 3 conjuntos de valores de colores
saved_colors = [None, None, None]
current_save_index = 0

# Variables globales para almacenar un solo valor de color
saved_colors1 = [None]
current_save_index1 = 0

# Variable global para controlar el estado del bucle (modo de guardado)
mode = 0  # 0 para guardar hasta 3 valores, 1 para guardar 1 valor

def get_color_from_image(event, x, y, flags, param):
    global selected_colors, hsv_image
    if event == cv2.EVENT_LBUTTONDOWN:
        pixel = hsv_image[y, x]
        selected_hue = pixel[0]
        selected_saturation = pixel[1]
        selected_value = pixel[2]
        
        # Agrega los valores seleccionados a la lista
        selected_colors.append((selected_hue, selected_saturation, selected_value))
        
        # Calcula los rangos
        if mode == 0:
            lower_red, upper_red = calculate_ranges(selected_hue, selected_saturation, selected_value)
        else:
            lower_red, upper_red = calculate_rangess(selected_hue, selected_saturation, selected_value)
            #print(f'{lower_red[0]},{lower_red[1]},{lower_red[2]},{upper_red[0]},{upper_red[1]},{upper_red[2]}')

def calculate_ranges(hue, saturation, value):
#    lower_red = np.array([max(hue-50, 0), max(saturation-50, 0), max(value-10, 0)])
#    upper_red = np.array([min(hue+100, 180), min(saturation+20, 255), min(value+10, 255)])
#    return lower_red, upper_red
    lower_red = np.array([max(hue-10, 0), max(saturation-50, 0), max(value-50, 0)])
    upper_red = np.array([min(hue+10, 180), min(saturation+200, 255), min(value+200, 255)])
    return lower_red, upper_red
def calculate_rangess(hue, saturation, value):
    lower_red = np.array([max(hue-10, 0), max(saturation-50, 0), max(value-50, 0)])
    upper_red = np.array([min(hue+180, 180), min(saturation+200, 255), min(value+200, 255)])
    return lower_red, upper_red

def normalize_color(image, hue, saturation, value):
    global hsv_image
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    if mode == 0:
        lower_red, upper_red = calculate_ranges(hue, saturation, value)
    else:
        lower_red, upper_red = calculate_rangess(hue, saturation, value)
    mask = cv2.inRange(hsv_image, lower_red, upper_red)
    normalized_color = cv2.bitwise_and(image, image, mask=mask)
    return normalized_color

def cargar_y_preparar_imagen(ruta_imagen):
    global hsv_image
    global scaled_image
    image = cv2.imread(ruta_imagen)
    scaled_image = cv2.resize(image, (0, 0), fx=1.0, fy=1.0)
    hsv_image = cv2.cvtColor(scaled_image, cv2.COLOR_BGR2HSV)

def bucle_principal(ruta_imagen):
    global selected_colors, saved_colors, current_save_index, mode
    global saved_colors1, current_save_index1
    
    cargar_y_preparar_imagen(ruta_imagen)
    
    cv2.namedWindow('Imagen Original')
    cv2.setMouseCallback('Imagen Original', get_color_from_image)
    
    while True:
        cv2.imshow('Imagen Original', scaled_image)
        
        if len(selected_colors) > 0:
            hue, saturation, value = selected_colors[-1]
            normalized_image = normalize_color(scaled_image, hue, saturation, value)
            cv2.imshow('Color Normalizado', normalized_image)

        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # Tecla ESC
            if mode == 0:
                # Imprime los valores guardados al cerrar
                print("Valores guardados en total:")
                for index, values in enumerate(saved_colors):
                    if values is not None:
                        #print(f'Posición {index + 1} - Inferior: H: {values[0]}, S: {values[1]}, V: {values[2]} | Superior: H: {values[3]}, S: {values[4]}, V: {values[5]}')
                        print(f'{values[0]},{values[1]},{values[2]},{values[3]},{values[4]},{values[5]}')
                # Cambia al modo de guardar solo un valor
                mode = 1
                selected_colors.clear()
                current_save_index = 0
                saved_colors = [None, None, None]
                continue
            else:
                break
        elif key == ord('v'):  # Letra 'v'
            if mode == 0:
                if len(selected_colors) > 0 and current_save_index < 3:
                    hue, saturation, value = selected_colors[-1]
                    lower_red, upper_red = calculate_ranges(hue, saturation, value)
                    saved_colors[current_save_index] = (
                        lower_red[0], lower_red[1], lower_red[2],
                        upper_red[0], upper_red[1], upper_red[2]
                    )
                    #print(f'Valores guardados en la posición {current_save_index + 1}: {lower_red[0]},{lower_red[1]},{lower_red[2]},{upper_red[0]},{upper_red[1]},{upper_red[2]}')
                    #print(f'{lower_red[0]},{lower_red[1]},{lower_red[2]},{upper_red[0]},{upper_red[1]},{upper_red[2]}') 
                    current_save_index += 1
            else:
                if len(selected_colors) > 0 and current_save_index1 < 1:
                    hue, saturation, value = selected_colors[-1]
                    lower_red, upper_red = calculate_rangess(hue, saturation, value)
                    saved_colors1[current_save_index1] = (
                        lower_red[0], lower_red[1], lower_red[2],
                        upper_red[0], upper_red[1], upper_red[2]
                    )
                    #print(f'Valores guardados en la posición {current_save_index1 + 1}: {lower_red[0]},{lower_red[1]},{lower_red[2]},{upper_red[0]},{upper_red[1]},{upper_red[2]}')
                    #print(f'{lower_red[0]},{lower_red[1]},{lower_red[2]},{upper_red[0]},{upper_red[1]},{upper_red[2]}')
                    current_save_index1 += 1
    
    # Imprime los valores guardados al cerrar
    if mode == 1:
        print("Valor guardado:")
        for index, values in enumerate(saved_colors1):
            if values is not None:
                #print(f'Posición {index + 1} - Inferior: H: {values[0]}, S: {values[1]}, V: {values[2]} | Superior: H: {values[3]}, S: {values[4]}, V: {values[5]}')
                print(f'{values[0]},{values[1]},{values[2]},{values[3]},{values[4]},{values[5]}')
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Define la ruta de la imagen
    #ruta_imagen = r'C:\Users\Vegan\OneDrive\Escritorio\GOTEOOOOOOOOOOOO\asd.jpg'
    ruta_imagen = r'C:\Users\Vegan\OneDrive\Escritorio\GOTEOOOOOOOOOOOO\homero.jpeg'
    # Crea el directorio si no existe
    os.makedirs(os.path.dirname(ruta_imagen), exist_ok=True)

    # Abre la cámara
    captura = cv2.VideoCapture(1)

    if not captura.isOpened():
        print("No se pudo abrir la cámara.")
        exit()

    # Captura una imagen
    ret, frame = captura.read()

    if not ret:
        print("No se pudo capturar la imagen.")
        captura.release()
        exit()

    # Guarda la imagen en la ruta especificada
    if cv2.imwrite(ruta_imagen, frame):
        print(f"Imagen guardada en: {ruta_imagen}")
    else:
        print("No se pudo guardar la imagen.")
    
    # Llama a la función principal con la ruta de la imagen
    bucle_principal(ruta_imagen)
    
    # Cierra la cámara
    captura.release()
    cv2.destroyAllWindows()

# SELECCIONAR LOS COLORES EN EL SIGUIENTE ORDEN TRAS APRETAR "V" PARA SU CONFIRMACION:
# 1RO: AMARILLO
# 2DO: AZUL
# 3RO: VERDE
# 4TO: VERDE Y AZUL (SELECCIONAR EL COLOR VERDE PARA DETECTAR EN CONJUNTO AMBOS)

Imagen guardada en: C:\Users\Vegan\OneDrive\Escritorio\GOTEOOOOOOOOOOOO\homero.jpeg
Valores guardados en total:
16,105,88,36,255,255
99,205,118,119,255,255
82,175,61,102,255,255
Valor guardado:
82,172,67,180,255,255


<span style="color:orange; font-size:32px; font-family:'Times New Roman'">**CODIGOS DE LAS FUNCIONES UTILIZADAS**</span>

<!-- 

color: cambia el color del texto 
font-size: establece el tamaño de la letra en píxeles.
font-family:'Times New Roman' cambia el tipo de letra a "Times New Roman".
** se utiliza para hacer que el texto del título esté en negrita

-->

In [33]:

# CARGAR LOS VALORES OBTENIDOS POR GOTEO AQUI:
Am1, Am2, Am3, Am4, Am5, Am6 =  [16,105,88,36,255,255]   # Amarillo
Az1, Az2, Az3, Az4, Az5, Az6 = [99,205,118,119,255,255] # Azul
Ve1, Ve2, Ve3, Ve4, Ve5, Ve6 = [82,175,61,102,255,255]     # Verde
AV1, AV2, AV3, AV4, AV5, AV6 = [82,172,67,180,255,255]    # Verde y Azul


#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGD              DETECCION DEL TABLERO, CAJA Y PIEZAS DE AJEDREZ                   GDGDGDGDGD    
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD

#------------------------------------[ Importamos las Librerias ]-----------------------------------------#

#*********************************************************************************************************#
import cv2                                  # Biblioteca para Tareas de Vision por Computadora            #
import numpy as np                          # Biblioteca para Tareas de Calculo Numerico                  #
from matplotlib import pyplot as plt        # Biblioteca para Tareas de Visualizacion de Datos y Graficos #
import chess                                                                                              #
import copy
#*********************************************************************************************************#

def deteccion (lower_h, lower_s, lower_v, upper_h, upper_s, upper_v, image, nombre_variable):
    #--------------------------------[Procesamiento de la imagen recortada]-----------------------------------#
    #image =  cv2.imread(r'C:\Users\Vegan\PROYECTO BRAZO ROBOT AJEDRECISTA\gaston.jpg')
    imagen_copia = image.copy()
    imagen1 = image.copy()
    #image = cv2.imread("Tablero de ajedrez.png")                                        # Cargo la imagen del tablero de ajedrez desde el archivo "Tablero de ajedrez.png", en la variable image
    
    hsv_image = cv2.cvtColor(imagen_copia, cv2.COLOR_BGR2HSV)                           # Convierto la imagen del espacio de color BGR (Azul, Verde, Rojo) a HSV (Hue, Saturation, Value). El espacio de color HSV separa la información de color (tono), la saturación (intensidad del color) y el valor (brillo) de manera más efectiva que el BGR

    lower_red = np.array([lower_h, lower_s, lower_v])                                   # Defino el rango de colores que quiero detectar en la imagen en formato HSV (Debe ajustarse segun sea necesario) . lower_red define el valor mínimo (Hue, Saturation, Value) y upper_red define el valor máximo para la detección                       
    upper_red = np.array([upper_h, upper_s, upper_v])

    mask = cv2.inRange(hsv_image, lower_red, upper_red)                                 # Creo una máscara binaria utilizando el rango de colores definido anteriormente.La mascara mostrara en blanco (255) lo que esta en el rango de colores y negro (0) lo que no
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)    # Busco los contornos de los objetos detectados en la máscara binaria.cv2.RETR_EXTERNAL devuelve solo los contornos externos (los bordes de los objetos), mientras que cv2.CHAIN_APPROX_SIMPLE comprime los contornos, eliminando los puntos redundantes y ahorrando memoria.
    nombre_variable = []                                                                # Inicializo un vector para almacenar las coordenadas de los centros de los contornos detectados.

    #-------------------------[Graficas del procesamiento de la imagen recortada]------------------------------#

    plt.figure(figsize=(10, 10))                                                        # Establecemos el tamaño de la figura en la que se mostrarán las imágenes (Pulgadas)

    plt.subplot(2, 2, 1)                                                                # Creamos un subplot en la primera fila y primera columna.
    plt.imshow(hsv_image)                                                               # Mostramos la imagen del espacio HSV
    plt.title('Imagen en espacio de color HSV')                                         # Establecemos el título del subplot

    plt.subplot(2, 2, 2)                                                                # Creamos un subplot en la primera fila y segunda columna
    plt.imshow(mask, cmap='gray')                                                       # Mostramos la imagen de la mascara binaria
    plt.title('Máscara binaria')                                                        # Establecemos el título del subplot

    masked_image = cv2.bitwise_and(imagen_copia, imagen_copia, mask=mask)               # cv2.bitwise_and() combina dos imágenes utilizando una máscara para determinar qué píxeles se mantienen y cuáles se descartan en la imagen resultante
    plt.subplot(2, 2, 3)                                                                # Creamos un subplot en la segunda fila y primera columna.
    plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB))                           # Mostramos la imagen original con la mascara,"cv2-cvtcolor" convierte la imagen masked_image del espacio de color BGR (Blue, Green, Red) al espacio de color RGB (Red, Green, Blue)
    plt.title('Imagen original con la máscara binaria aplicada')                        # Establecemos el título del subplot

    contour_image = cv2.drawContours(masked_image.copy(), contours, -1, (255, 255, 255), 1) # cv2.drawContours() modifica la imagen sobre la que se dibujan los contornos. Al crear una copia de la imagen original, se conserva la imagen original intacta
    plt.subplot(2, 2, 4)                                                                # Creamos un subplot en la segunda fila y segunda columna
    plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))                          # Mostramos la imagen original con contornos
    plt.title('Imagen con contornos')                                                   # Establecemos el título del subplot

    plt.tight_layout()                                                                  # Ajustamos automáticamente la disposición de los subplots para evitar superposiciones y mejorando la visualizacion
    plt.show()                                                                          # Mostramos la figura con todos los subplots

    #-------------------------[Obtencion de coordenadas de las esquinas del tablero]------------------------------#

    for contour in contours:                                                            # Recorro cada contorno encontrado para calcular su centro y dibujar un círculo en ese punto.                                              
        moments = cv2.moments(contour)                                                  # Calculo los momentos de masa del contorno para encontrar su centro
        if moments["m00"] > 30:                                                         # Compruebo que el área del contorno no sea cero para evitar divisiones por cero.
            cx = int(moments["m10"] / moments["m00"])                                   # Coordenada x del centro del contorno.
            cy = int(moments["m01"] / moments["m00"])                                   # Coordenada y del centro del contorno.
            nombre_variable.append((cx, cy))                                            # Añado las coordenadas del centro al vector de coordenadas.

            cv2.circle(imagen_copia, (cx, cy), 5, (0, 255, 0), -1)                                                                 # Se dibuja un círculo en la imagen original image en las coordenadas del centro del contorno. El círculo tiene un radio de 5 píxeles y está coloreado en verde (0, 255, 0).
            cv2.putText(imagen_copia, f'({cx}, {cy})', (cx - 30 , cy -10 ), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)     # Se añade un texto con las coordenadas (cx, cy) del centro del contorno. El texto se posiciona en las coordenadas (cx - 30, cy - 10) con una fuente cv2.FONT_HERSHEY_SIMPLEX, tamaño de fuente 0.4, color blanco (255, 255, 255) y grosor 2.

    print("Vector de coordenadas:", nombre_variable)
    print("Cantidad de coordenadas:",len(nombre_variable))
    plt.figure(figsize=(10, 10))                                                          # Establecemos el tamaño de la figura en la que se mostrarán las imágen(Pulgadas)
    plt.title('Imagen con coordenadas resultante')    
    # Establecemos el título de plot
    plt.imshow(cv2.cvtColor(imagen_copia, cv2.COLOR_BGR2RGB))                           # Mostramos la imagen original con las coordenadas resultantes
    plt.show()                                                                          # Mostramos la figura 

# Divide las coordenadas en x_coords e y_coords
    x_coords, y_coords = zip(*nombre_variable)
    
    return x_coords, y_coords, imagen1, nombre_variable


#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#

     #´´´´´´´´´´´´´´´´´[Creo la CAJA a partir de las 4 coordenadas obtenidas]´´´´´´´´´´´´´´´´´´´´#
#´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´[Genero mi Tablero-Matriz de coordenadas]´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´#
def todas_las_coordenadas(x_coords, y_coords):
    # Creo el cuadrado a partir de las coordenadas
    x, y = x_coords, y_coords
    min_x, max_x = min(x_coords), max(x_coords)
    min_y, max_y = min(y), max(y)

    # Tamaño de cada casilla
    tamaño_casilla_x = (max_x - min_x) / 4  #Como la coordenada no empieza en cero, le resto min_x
    tamaño_casilla_y = (max_y - min_y) / 9  #Como la coordenada no empieza en cero, le resto min_y

    # Creo la Matriz de esquinas
    esquinas = np.zeros((5, 10, 2))# Matriz de 9x4 para almacenar las coordenadas que contiene 2 elementos (x,y)
    # Asigno las coordenadas de las esquinas de cada casilla en la matriz esquinas mediante dos ciclo for
    for i in range(5):     # i representa el índice de la fila  
        for j in range(10):  # j representa el índice de la columna.
            esquinas[i, j] = [min_x + i * tamaño_casilla_x, min_y + j * tamaño_casilla_y]

    esquinas_1 = esquinas
    return esquinas_1

#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#

def casillas_y_sus_puntos_medios(esquinas,imagen1):
    matriz = esquinas
    celdas = [] 
    # Recorro la matriz 
    for fila in range(4):
        for columna in range(9):
            # Guardo las coordenadas de las esquinas de las celdas
            esquina_inf_izq = matriz[fila][columna]    
            esquina_inf_der = matriz[fila][columna + 1] 
            esquina_sup_izq = matriz[fila + 1][columna]
            esquina_sup_der = matriz[fila + 1][columna + 1]
        
            # Agrego las esquinas a la lista de celdas
            celdas.append((esquina_inf_izq, esquina_inf_der, esquina_sup_izq, esquina_sup_der))

#´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´[Obtengo el centro de cada Casilla]´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´#

    # Coordenadas de todas las celdas
    for i, celda in enumerate(celdas):
    #enumerate(celdas) nos permite obtener tanto el índice como el valor de cada elemento en la lista
    #i representa el índice de la celda en la lista.
    #celda representa el valor de la celda actual en la lista celdas
        x_sum = 0
        y_sum = 0
        for esquina in celda:
            x, y = esquina       #Desempaquetamos las coordenadas x e y de la esquina actual
            # Acumulamos las coordenadas x e y de todas las esquinas de la celda.
            x_sum += x           #Es una forma abreviada de escribir x_sum = x_sum + x
            y_sum += y           #En cada iteración, y_sum se actualiza sumando el valor actual de y, igual con x
        # En cada celda tengo 4 coordenadas,
        # pues aca estoy dividiendo la suma total de esas 4 coordeadas dividido 4
        x_medio = x_sum / len(celda)
        y_medio = y_sum / len(celda)
        #print(f"El punto medio de la celda {i+1} es ({x_medio}, {y_medio})")
        
    # Inicializo listas vacías para almacenar las coordenadas x e y
    x_coords = []
    y_coords = []

    # Extrigo las coordenadas x e y de cada punto 
    for fila in matriz:
        for p in fila:
            x, y = p
            x_coords.append(x)
            y_coords.append(y)
            
#´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´[Grafico Restante]´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´#

    # Superpongo las coordenadas en la imagen
    for x, y in zip(x_coords, y_coords):
        cv2.circle(imagen1, (int(x), int(y)), 3, (0, 255, 0), -1)  # Dibujo un círculo rojo en cada esquina
       # Muestro las coordenadas de cada esquina cerca del círculo.
        #cv2.putText(imagen1, f"({int(x)}, {int(y)})", (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.25, (255, 0, 0), 1)#####

    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(imagen1, cv2.COLOR_BGR2RGB))
    plt.title("Esquinas del tablero de Ajedrez")
    plt.axis("off")  # Oculta los ejes
    plt.show()

#########################################################################################################
#########################################################################################################

def detector_por_color_esquinas (lower_h, lower_s, lower_v, upper_h, upper_s, upper_v,image, nombre_variable):
    #--------------------------------[Procesamiento de la imagen recortada]-----------------------------------#

    #image = cv2.imread("Tablero de ajedrez.png")                                        # Cargo la imagen del tablero de ajedrez desde el archivo "Tablero de ajedrez.png", en la variable image
    imagen_copia = image.copy()                                                         # Creamos una copia de la imagen para agregar etiquetas sin alterar la original 

    hsv_image = cv2.cvtColor(imagen_copia, cv2.COLOR_BGR2HSV)                           # Convierto la imagen del espacio de color BGR (Azul, Verde, Rojo) a HSV (Hue, Saturation, Value). El espacio de color HSV separa la información de color (tono), la saturación (intensidad del color) y el valor (brillo) de manera más efectiva que el BGR

    lower_red = np.array([lower_h, lower_s, lower_v])                                   # Defino el rango de colores que quiero detectar en la imagen en formato HSV (Debe ajustarse segun sea necesario) . lower_red define el valor mínimo (Hue, Saturation, Value) y upper_red define el valor máximo para la detección                       
    upper_red = np.array([upper_h, upper_s, upper_v])

    mask = cv2.inRange(hsv_image, lower_red, upper_red)                                 # Creo una máscara binaria utilizando el rango de colores definido anteriormente.La mascara mostrara en blanco (255) lo que esta en el rango de colores y negro (0) lo que no
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)    # Busco los contornos de los objetos detectados en la máscara binaria.cv2.RETR_EXTERNAL devuelve solo los contornos externos (los bordes de los objetos), mientras que cv2.CHAIN_APPROX_SIMPLE comprime los contornos, eliminando los puntos redundantes y ahorrando memoria.
    nombre_variable = []                                                                # Inicializo un vector para almacenar las coordenadas de los centros de los contornos detectados.

    #-------------------------[Graficas del procesamiento de la imagen recortada]------------------------------#

    plt.figure(figsize=(10, 10))                                                        # Establecemos el tamaño de la figura en la que se mostrarán las imágenes (Pulgadas)

    plt.subplot(2, 2, 1)                                                                # Creamos un subplot en la primera fila y primera columna.
    plt.imshow(hsv_image)                                                               # Mostramos la imagen del espacio HSV
    plt.title('Imagen en espacio de color HSV')                                         # Establecemos el título del subplot

    plt.subplot(2, 2, 2)                                                                # Creamos un subplot en la primera fila y segunda columna
    plt.imshow(mask, cmap='gray')                                                       # Mostramos la imagen de la mascara binaria
    plt.title('Máscara binaria')                                                        # Establecemos el título del subplot

    masked_image = cv2.bitwise_and(imagen_copia, imagen_copia, mask=mask)               # cv2.bitwise_and() combina dos imágenes utilizando una máscara para determinar qué píxeles se mantienen y cuáles se descartan en la imagen resultante
    plt.subplot(2, 2, 3)                                                                # Creamos un subplot en la segunda fila y primera columna.
    plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB))                           # Mostramos la imagen original con la mascara,"cv2-cvtcolor" convierte la imagen masked_image del espacio de color BGR (Blue, Green, Red) al espacio de color RGB (Red, Green, Blue)
    plt.title('Imagen original con la máscara binaria aplicada')                        # Establecemos el título del subplot

    contour_image = cv2.drawContours(masked_image.copy(), contours, -1, (255, 255, 255), 1) # cv2.drawContours() modifica la imagen sobre la que se dibujan los contornos. Al crear una copia de la imagen original, se conserva la imagen original intacta
    plt.subplot(2, 2, 4)                                                                # Creamos un subplot en la segunda fila y segunda columna
    plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))                          # Mostramos la imagen original con contornos
    plt.title('Imagen con contornos')                                                   # Establecemos el título del subplot

    plt.tight_layout()                                                                  # Ajustamos automáticamente la disposición de los subplots para evitar superposiciones y mejorando la visualizacion
    plt.show()                                                                          # Mostramos la figura con todos los subplots

    #-------------------------[Obtencion de coordenadas de las esquinas del tablero]------------------------------#

    for contour in contours:                                                            # Recorro cada contorno encontrado para calcular su centro y dibujar un círculo en ese punto.                                              
        moments = cv2.moments(contour)                                                  # Calculo los momentos de masa del contorno para encontrar su centro
        if moments["m00"] > 30:                                                         # Compruebo que el área del contorno no sea cero para evitar divisiones por cero.
            cx = int(moments["m10"] / moments["m00"])                                   # Coordenada x del centro del contorno.
            cy = int(moments["m01"] / moments["m00"])                                   # Coordenada y del centro del contorno.
            nombre_variable.append((cx, cy))                                            # Añado las coordenadas del centro al vector de coordenadas.

            cv2.circle(imagen_copia, (cx, cy), 5, (0, 255, 0), -1)                                                                 # Se dibuja un círculo en la imagen original image en las coordenadas del centro del contorno. El círculo tiene un radio de 5 píxeles y está coloreado en verde (0, 255, 0).
            cv2.putText(imagen_copia, f'({cx}, {cy})', (cx - 30 , cy -10 ), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)     # Se añade un texto con las coordenadas (cx, cy) del centro del contorno. El texto se posiciona en las coordenadas (cx - 30, cy - 10) con una fuente cv2.FONT_HERSHEY_SIMPLEX, tamaño de fuente 0.4, color blanco (255, 255, 255) y grosor 2.

    # Encontrar las coordenadas máximas y mínimas en X e Y
    min_x = min(nombre_variable, key=lambda x: x[0])[0]
    max_x = max(nombre_variable, key=lambda x: x[0])[0]
    min_y = min(nombre_variable, key=lambda x: x[1])[1]
    max_y = max(nombre_variable, key=lambda x: x[1])[1]

    # Determinar las esquinas según las coordenadas máximas y mínimas
    #esquina_inf_izquierda = (min_x, max_y)
    #esquina_inf_derecha = (max_x, max_y)
    #esquina_sup_derecha = (max_x, min_y)
    #esquina_sup_izquierda = (min_x, min_y)

    # Crear el vector de coordenadas ordenado según el orden requerido
    nombre_variable = [(min_x, max_y), (max_x, max_y), (max_x, min_y), (min_x, min_y)]      
    print("Vector de coordenadas:", nombre_variable)
    print("Cantidad de coordenadas:",len(nombre_variable))
    plt.figure(figsize=(5, 5))                                                          # Establecemos el tamaño de la figura en la que se mostrarán las imágen(Pulgadas)
    plt.title('Imagen con coordenadas resultante')                                      # Establecemos el título de plot
    
    plt.imshow(cv2.cvtColor(imagen_copia, cv2.COLOR_BGR2RGB))                           # Mostramos la imagen original con las coordenadas resultantes
    plt.show()                                                                          # Mostramos la figura 

    return nombre_variable

#########################################################################################################
def detector_por_color (lower_h, lower_s, lower_v, upper_h, upper_s, upper_v,image, nombre_variable):
    #--------------------------------[Procesamiento de la imagen recortada]-----------------------------------#

    #image = cv2.imread("Tablero de ajedrez.png")                                        # Cargo la imagen del tablero de ajedrez desde el archivo "Tablero de ajedrez.png", en la variable image
    imagen_copia = image.copy()                                                         # Creamos una copia de la imagen para agregar etiquetas sin alterar la original 

    hsv_image = cv2.cvtColor(imagen_copia, cv2.COLOR_BGR2HSV)                           # Convierto la imagen del espacio de color BGR (Azul, Verde, Rojo) a HSV (Hue, Saturation, Value). El espacio de color HSV separa la información de color (tono), la saturación (intensidad del color) y el valor (brillo) de manera más efectiva que el BGR

    lower_red = np.array([lower_h, lower_s, lower_v])                                   # Defino el rango de colores que quiero detectar en la imagen en formato HSV (Debe ajustarse segun sea necesario) . lower_red define el valor mínimo (Hue, Saturation, Value) y upper_red define el valor máximo para la detección                       
    upper_red = np.array([upper_h, upper_s, upper_v])

    mask = cv2.inRange(hsv_image, lower_red, upper_red)                                 # Creo una máscara binaria utilizando el rango de colores definido anteriormente.La mascara mostrara en blanco (255) lo que esta en el rango de colores y negro (0) lo que no
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)    # Busco los contornos de los objetos detectados en la máscara binaria.cv2.RETR_EXTERNAL devuelve solo los contornos externos (los bordes de los objetos), mientras que cv2.CHAIN_APPROX_SIMPLE comprime los contornos, eliminando los puntos redundantes y ahorrando memoria.
    nombre_variable = []                                                                # Inicializo un vector para almacenar las coordenadas de los centros de los contornos detectados.

    #-------------------------[Graficas del procesamiento de la imagen recortada]------------------------------#

    plt.figure(figsize=(10, 10))                                                        # Establecemos el tamaño de la figura en la que se mostrarán las imágenes (Pulgadas)

    plt.subplot(2, 2, 1)                                                                # Creamos un subplot en la primera fila y primera columna.
    plt.imshow(hsv_image)                                                               # Mostramos la imagen del espacio HSV
    plt.title('Imagen en espacio de color HSV')                                         # Establecemos el título del subplot

    plt.subplot(2, 2, 2)                                                                # Creamos un subplot en la primera fila y segunda columna
    plt.imshow(mask, cmap='gray')                                                       # Mostramos la imagen de la mascara binaria
    plt.title('Máscara binaria')                                                        # Establecemos el título del subplot

    masked_image = cv2.bitwise_and(imagen_copia, imagen_copia, mask=mask)               # cv2.bitwise_and() combina dos imágenes utilizando una máscara para determinar qué píxeles se mantienen y cuáles se descartan en la imagen resultante
    plt.subplot(2, 2, 3)                                                                # Creamos un subplot en la segunda fila y primera columna.
    plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB))                           # Mostramos la imagen original con la mascara,"cv2-cvtcolor" convierte la imagen masked_image del espacio de color BGR (Blue, Green, Red) al espacio de color RGB (Red, Green, Blue)
    plt.title('Imagen original con la máscara binaria aplicada')                        # Establecemos el título del subplot

    contour_image = cv2.drawContours(masked_image.copy(), contours, -1, (255, 255, 255), 1) # cv2.drawContours() modifica la imagen sobre la que se dibujan los contornos. Al crear una copia de la imagen original, se conserva la imagen original intacta
    plt.subplot(2, 2, 4)                                                                # Creamos un subplot en la segunda fila y segunda columna
    plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))                          # Mostramos la imagen original con contornos
    plt.title('Imagen con contornos')                                                   # Establecemos el título del subplot
    plt.tight_layout()                                                                  # Ajustamos automáticamente la disposición de los subplots para evitar superposiciones y mejorando la visualizacion
    plt.show()                                                                          # Mostramos la figura con todos los subplots

    #-------------------------[Obtencion de coordenadas de las esquinas del tablero]------------------------------#

    for contour in contours:                                                            # Recorro cada contorno encontrado para calcular su centro y dibujar un círculo en ese punto.                                              
        moments = cv2.moments(contour)                                                  # Calculo los momentos de masa del contorno para encontrar su centro
        if moments["m00"] > 30:                                                         # Compruebo que el área del contorno no sea cero para evitar divisiones por cero.
            cx = int(moments["m10"] / moments["m00"])                                   # Coordenada x del centro del contorno.
            cy = int(moments["m01"] / moments["m00"])                                   # Coordenada y del centro del contorno.
            nombre_variable.append((cx, cy))                                            # Añado las coordenadas del centro al vector de coordenadas.

            cv2.circle(imagen_copia, (cx, cy), 5, (0, 255, 0), -1)                                                                 # Se dibuja un círculo en la imagen original image en las coordenadas del centro del contorno. El círculo tiene un radio de 5 píxeles y está coloreado en verde (0, 255, 0).
            cv2.putText(imagen_copia, f'({cx}, {cy})', (cx - 30 , cy -10 ), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)     # Se añade un texto con las coordenadas (cx, cy) del centro del contorno. El texto se posiciona en las coordenadas (cx - 30, cy - 10) con una fuente cv2.FONT_HERSHEY_SIMPLEX, tamaño de fuente 0.4, color blanco (255, 255, 255) y grosor 2.


    # Crear el vector de coordenadas ordenado según el orden requerido
    print("Vector de coordenadas:", nombre_variable)
    print("Cantidad de coordenadas:",len(nombre_variable))
    plt.figure(figsize=(5, 5))                                                          # Establecemos el tamaño de la figura en la que se mostrarán las imágen(Pulgadas)
    plt.title('Imagen con coordenadas resultante')                                      # Establecemos el título de plot
    plt.imshow(cv2.cvtColor(imagen_copia, cv2.COLOR_BGR2RGB))                           # Mostramos la imagen original con las coordenadas resultantes
    plt.show()                                                                          # Mostramos la figura 

    return nombre_variable

#########################################################################################################
#########################################################################################################
def detector_por_color2 (lower_h, lower_s, lower_v, upper_h, upper_s, upper_v,image, nombre_variable):
    #--------------------------------[Procesamiento de la imagen recortada]-----------------------------------#

    #image = cv2.imread("Tablero de ajedrez.png")                                        # Cargo la imagen del tablero de ajedrez desde el archivo "Tablero de ajedrez.png", en la variable image
    imagen_copia = image.copy()                                                         # Creamos una copia de la imagen para agregar etiquetas sin alterar la original 

    hsv_image = cv2.cvtColor(imagen_copia, cv2.COLOR_BGR2HSV)                           # Convierto la imagen del espacio de color BGR (Azul, Verde, Rojo) a HSV (Hue, Saturation, Value). El espacio de color HSV separa la información de color (tono), la saturación (intensidad del color) y el valor (brillo) de manera más efectiva que el BGR

    lower_red = np.array([lower_h, lower_s, lower_v])                                   # Defino el rango de colores que quiero detectar en la imagen en formato HSV (Debe ajustarse segun sea necesario) . lower_red define el valor mínimo (Hue, Saturation, Value) y upper_red define el valor máximo para la detección                       
    upper_red = np.array([upper_h, upper_s, upper_v])

    mask = cv2.inRange(hsv_image, lower_red, upper_red)                                 # Creo una máscara binaria utilizando el rango de colores definido anteriormente.La mascara mostrara en blanco (255) lo que esta en el rango de colores y negro (0) lo que no
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)    # Busco los contornos de los objetos detectados en la máscara binaria.cv2.RETR_EXTERNAL devuelve solo los contornos externos (los bordes de los objetos), mientras que cv2.CHAIN_APPROX_SIMPLE comprime los contornos, eliminando los puntos redundantes y ahorrando memoria.
    nombre_variable = []                                                                # Inicializo un vector para almacenar las coordenadas de los centros de los contornos detectados.

    #-------------------------[Graficas del procesamiento de la imagen recortada]------------------------------#

    #plt.figure(figsize=(10, 10))                                                        # Establecemos el tamaño de la figura en la que se mostrarán las imágenes (Pulgadas)

    #plt.subplot(2, 2, 1)                                                                # Creamos un subplot en la primera fila y primera columna.
    #plt.imshow(hsv_image)                                                               # Mostramos la imagen del espacio HSV
    #plt.title('Imagen en espacio de color HSV')                                         # Establecemos el título del subplot

    #plt.subplot(2, 2, 2)                                                                # Creamos un subplot en la primera fila y segunda columna
    #plt.imshow(mask, cmap='gray')                                                       # Mostramos la imagen de la mascara binaria
    #plt.title('Máscara binaria')                                                        # Establecemos el título del subplot

    #masked_image = cv2.bitwise_and(imagen_copia, imagen_copia, mask=mask)               # cv2.bitwise_and() combina dos imágenes utilizando una máscara para determinar qué píxeles se mantienen y cuáles se descartan en la imagen resultante
    #plt.subplot(2, 2, 3)                                                                # Creamos un subplot en la segunda fila y primera columna.
    #plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB))                           # Mostramos la imagen original con la mascara,"cv2-cvtcolor" convierte la imagen masked_image del espacio de color BGR (Blue, Green, Red) al espacio de color RGB (Red, Green, Blue)
    #plt.title('Imagen original con la máscara binaria aplicada')                        # Establecemos el título del subplot

    #contour_image = cv2.drawContours(masked_image.copy(), contours, -1, (255, 255, 255), 1) # cv2.drawContours() modifica la imagen sobre la que se dibujan los contornos. Al crear una copia de la imagen original, se conserva la imagen original intacta
    #plt.subplot(2, 2, 4)                                                                # Creamos un subplot en la segunda fila y segunda columna
    #plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))                          # Mostramos la imagen original con contornos
    #plt.title('Imagen con contornos')                                                   # Establecemos el título del subplot

    #plt.tight_layout()                                                                  # Ajustamos automáticamente la disposición de los subplots para evitar superposiciones y mejorando la visualizacion
    #plt.show()                                                                          # Mostramos la figura con todos los subplots

    #-------------------------[Obtencion de coordenadas de las esquinas del tablero]------------------------------#

    for contour in contours:                                                            # Recorro cada contorno encontrado para calcular su centro y dibujar un círculo en ese punto.                                              
        moments = cv2.moments(contour)                                                  # Calculo los momentos de masa del contorno para encontrar su centro
        if moments["m00"] > 30:                                                         # Compruebo que el área del contorno no sea cero para evitar divisiones por cero.
            cx = int(moments["m10"] / moments["m00"])                                   # Coordenada x del centro del contorno.
            cy = int(moments["m01"] / moments["m00"])                                   # Coordenada y del centro del contorno.
            nombre_variable.append((cx, cy))                                            # Añado las coordenadas del centro al vector de coordenadas.

            cv2.circle(imagen_copia, (cx, cy), 5, (0, 255, 0), -1)                                                                 # Se dibuja un círculo en la imagen original image en las coordenadas del centro del contorno. El círculo tiene un radio de 5 píxeles y está coloreado en verde (0, 255, 0).
            cv2.putText(imagen_copia, f'({cx}, {cy})', (cx - 30 , cy -10 ), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)     # Se añade un texto con las coordenadas (cx, cy) del centro del contorno. El texto se posiciona en las coordenadas (cx - 30, cy - 10) con una fuente cv2.FONT_HERSHEY_SIMPLEX, tamaño de fuente 0.4, color blanco (255, 255, 255) y grosor 2.

    #print("Vector de coordenadas:", nombre_variable)
    #print("Cantidad de coordenadas:",len(nombre_variable))
    #plt.figure(figsize=(5, 5))                                                          # Establecemos el tamaño de la figura en la que se mostrarán las imágen(Pulgadas)
    #plt.title('Imagen con coordenadas resultante')                                      # Establecemos el título de plot
    #plt.imshow(cv2.cvtColor(imagen_copia, cv2.COLOR_BGR2RGB))                           # Mostramos la imagen original con las coordenadas resultantes
    #plt.show()                                                                          # Mostramos la figura 

    return nombre_variable

#########################################################################################################
#########################################################################################################

def Armado_de_nodos_de_las_casillas(coordenadas_esquinas,image):
    #--------------------------------------[Division de casillas]---------------------------------------------#

    #image = cv2.imread("Tablero de ajedrez.png")                                            # Cargar la imagen del tablero de ajedrez nueva sin dibujos
    imagen_copia = image.copy()                                                              # Creamos una copia de la imagen para agregar etiquetas sin alterar la original 

    #print("Vector de coordenadas de las esquinas del tablero:", coordenadas)               # Ejemplo : Vector de coordenadas de las esquinas del tablero: [(9, 345), (350, 340), (346, 15), (12, 9)]

    x1 = coordenadas_esquinas[0][0]                                                         # Obtenemos la coordenada x de la esquina inferior izquierda
    x2 = coordenadas_esquinas[1][0]                                                         # Obtenemos la coordenada x de la esquina inferior derecha
    y1 = coordenadas_esquinas[0][1]                                                         # Obtenemos la coordenada y de la esquina inferior izquierda
    y4 = coordenadas_esquinas[3][1]                                                         # Obtenemos la coordenada y de la esquina superior izquierda

    dif_horizontal = np.abs(x2 - x1)                                                        # Ejemplo : |(350)-(9)| --> 341     # Calculamos la diferencia horizontal y vertical entre las esquinas
    dif_vertical = np.abs(y4 - y1)                                                          # Ejemplo : |(9)-(345)| --> 336

    segmento_horizontal = dif_horizontal / 8                                                # Dividimos las diferencias entre 8 para obtener el tamaño de cada casilla del tablero
    segmento_vertical = dif_vertical / 8

    coordenadas_matriz = np.zeros((9, 9, 2), dtype=int)                                     # Creamos una matriz llena de ceros de 9 filas , 9 columnas y 2 profundidades (x,y). Los elementos de la matriz son enteros

    #-----------------------------[Imagen con puntos de la division de casillas]-------------------------------#

    for x in range(9):                                                                      # itero 9 veces, representando las divisiones horizontales del tablero.
        for y in range(9):                                                                  # itero 9 veces, representando las divisiones verticales del tablero.
            x_coord = int(x1 + x * segmento_horizontal)                                     # Se suma la coordenada de la esquina inferior izquierda (x1) con el producto del índice actual del bucle x y el segmento horizontal. El resultado se convierte a entero.
            y_coord = int(y1 - y * segmento_vertical)                                       # Se resta la coordenada de la esquina inferior izquierda (y1) con el producto del índice actual del bucle y y el segmento vertical.Se usa el signo negativo para ajustar la dirección hacia arriba. El resultado se convierte en entero  
            coordenadas_matriz[x, y] = [x_coord, y_coord]                                   # Agregar las coordenadas al vector
            cv2.circle(imagen_copia, (x_coord, y_coord), 3, (0, 255, 0), -1)                # Dibujamos un punto en la(imagen_copia) en las coordenadas (x_coord, y_coord). El punto tiene un radio de 3 píxeles, color verde y el -1 indica que se rellena el circulo.

    plt.figure(figsize=(6, 6))                                                              # Establecemos el tamaño de la figura en la que se mostrará la imágen (Pulgadas)
    plt.title('Tablero de ajedrez con coordenadas 9x9')                                     # Establecemos el título de plot
    plt.imshow(cv2.cvtColor(imagen_copia, cv2.COLOR_BGR2RGB))                               # Mostramos la imagen original (pero copia) con las coordenadas resultantes
    plt.show()                                                                              # Mostramos la figura 

    MatrizTraspuesta = coordenadas_matriz.transpose(1,0,2)                                  # transpose() permuta las dimensiones de un array. (1,0,2) la segunda dimension se mueva a la primera posicion, la primera dimension se mueve a la segunda posicion, y hay 2 profundidades (x,y)                         

    print("Matriz completa de cada coordenada del tablero:\n\n",MatrizTraspuesta.flatten(),"\n") # numpy.ndarray.flatten() es para aplanar la matriz y luego imprimir los elementos uno al lado del otro. 
    #print("fila 1:\n\n", coordenadas_matriz.transpose(1,0,2) [0,:])                        # Por si quisiera ver alguna fila o columna en especifico
    #print("columna 1:\n\n", coordenadas_matriz.transpose(1,0,2) [:,0])

    return MatrizTraspuesta

#########################################################################################################
#########################################################################################################

def Etiquetado_de_piezas_en_cada_casilla(coordenadas_piezas,MatrizTraspuesta):
    
    #---------------------------[Etiquetado de cada casilla del tablero]-------------------------------#
    Tablero_inicial =[['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r'],
                    ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
                    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
                    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
                    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
                    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
                    ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
                    ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']]

    def obtener_pieza(coordenada):
        fila, columna = coordenada
        return Tablero_inicial[fila][columna]

    matriz_tablero = np.empty((8, 8), dtype=str)                                            # Creamos una matriz de 8 filas y 8 columnas. El argumento dtype=str especifica que los elementos de la matriz serán de tipo string                                                                                                
    matriz_tablero.fill(" ")                                                                # llenamos todo el array con un valor específico. En este caso, estamos llenando la matriz con el carácter "-",para una mejor visualizacion.

    for k in coordenadas_piezas:                                                                                                                        # Este ciclo for sirve para recorrer el vector de coordenadas de las piezaas detectadas k[0] me da todas las del eje "x" , k[1] me da todas las del eje "y"
        for i in range(8):                                                                                                                              # Este ciclo for es para recorrer la matriz traspuesta en el eje horizontal, osea las filas
            for j in range(8):                                                                                                                          # Este ciclo for es para recorrer la matriz traspuesta en el eje vertical, osea las columnas
                letra = chr(65 +j)                                                                                                                      # Convertir número a letra (A=65, B=66, ...). Esto es para la denominacion del tablero que va desde la A hasta la H
                if (MatrizTraspuesta[0,j,0] <= k[0] <= MatrizTraspuesta[0,j+1,0]) and (MatrizTraspuesta[i,j,1] >= k[1] >= MatrizTraspuesta[i+1,j,1]):   # Si se cumplen 2 condiciones juntas, tanto para la coordenada en "x" como la coordenada en "y", de nuestro vector de coordenadas puedo saber en que casilla esta ubicado
                    pieza = obtener_pieza((7 - i, j))
                    print(f"Hay una pieza {pieza} en la casilla {letra}{i+1} y su coordenada es {k}")                                                   # Imprimo el mensaje de la casilla y la coordenada que tiene
                    matriz_tablero[7-i][j] = pieza                                                                                                      # La colocacion de la X en cada parte de la matriz. (La primera coordenada la invertimos debido a nuestra covencion de tablero)

    #print("\nMatriz del tablero con las piezas marcadas:\n")
    #print(matriz_tablero)
    return matriz_tablero
#########################################################################################################
#GDGDGDGDGDGDGDGDGDGDGDDGGDGDGDGDGDGDGDGDGDGDGDGDDGGDGDGDGDGDGDGDGDGDGDGGDGDGDGDGDGDGDGDGDGGDGDGDGDGDG
#GDGDGDGDGDGDGDGDGDGDGDDGGDGDGDGDGDGDGDGDGDGDGDGDDGGDGDGDGDGDGDGDGDGDGDGGDGDGDGDGDGDGDGDGDGGDGDGDGDGDG
#GDGDGDGDGDGDGDGDGDGDGDDGGDGDGDGDGDGDGDGDGDGDGDGDDGGDGDGDGDGDGDGDGDGDGDGGDGDGDGDGDGDGDGDGDGGDGDGDGDGDG
#GDGDGDGDGDGDGDGDGDGDGDDGGDGDGDGDGDGDGDGDGDGDGDGDDGGDGDGDGDGDGDGDGDGDGDGGDGDGDGDGDGDGDGDGDGGDGDGDGDGDG
#GDGDGDGDGDGDGDGDGDGDGDDGGDGDGDGDGDGDGDGDGDGDGDGDDGGDGDGDGDGDGDGDGDGDGDGGDGDGDGDGDGDGDGDGDGGDGDGDGDGDG
#########################################################################################################
def detectar_nuevas_posiciones(coordenadas_esquinas, frame):
    # Detecta las piezas en conjunto tanto las verdes como las azules
    coordenadas_piezas_nuevas = detector_por_color(AV1, AV2, AV3, AV4, AV5, AV6 ,frame, nombre_variable = "coordenadas_piezas")
    return coordenadas_piezas_nuevas

#<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<

def etiquetar_piezas_actuales(coordenadas_piezas, matriz_traspuesta):
    # Creo una matriz vacía para representar el tablero actual
    matriz_tablero_actual = [[' ' for _ in range(8)] for _ in range(8)]
    
    # Itero sobre las coordenadas de las piezas detectadas
    for k in coordenadas_piezas:
        # Itero sobre las filas y columnas de la matriz traspuesta
        for i in range(8):
            for j in range(8):
                # Si la coordenada de la pieza está dentro de los límites de una casilla...
                if (matriz_traspuesta[0, j, 0] <= k[0] <= matriz_traspuesta[0, j + 1, 0]) and (
                        matriz_traspuesta[i, j, 1] >= k[1] >= matriz_traspuesta[i + 1, j, 1]):
                    # Marco la posición actual de la pieza en la matriz del tablero actual
                    matriz_tablero_actual[7 - i][j] = '-'

    #print("\nMatriz del tablero actual con las piezas marcadas:\n")
    #for fila in matriz_tablero_actual:
    #    print(' '.join(fila))
    return matriz_tablero_actual

#<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3

def letras(tablero_actual, tablero_inicial, turno):
    letra = ' '  # Inicializo la letra como un espacio vacío
    jugador_actual = "Blancas" if turno % 2 == 1 else "Negras"

    for i in range(8):
        for j in range(8):
            casilla_actual = tablero_actual[i][j]
            casilla_inicial = tablero_inicial[i][j]
            if casilla_actual != casilla_inicial:
                # Verifico las letras según el turno
                if jugador_actual == "Blancas":
                    if casilla_inicial == 'P' and casilla_actual == ' ':
                        letra = 'P'
                    elif casilla_inicial == 'R' and casilla_actual == ' ':
                        letra = 'R'
                    elif casilla_inicial == 'N' and casilla_actual == ' ':
                        letra = 'N'
                    elif casilla_inicial == 'B' and casilla_actual == ' ':
                        letra = 'B'
                    elif casilla_inicial == 'Q' and casilla_actual == ' ':
                        letra = 'Q'
                    elif casilla_inicial == 'K' and casilla_actual == ' ':
                        letra = 'K'
                else:  # Turno de las Negras
                    if casilla_inicial == 'p' and casilla_actual == ' ':
                        letra = 'p'
                    elif casilla_inicial == 'r' and casilla_actual == ' ':
                        letra = 'r'
                    elif casilla_inicial == 'n' and casilla_actual == ' ':
                        letra = 'n'
                    elif casilla_inicial == 'b' and casilla_actual == ' ':
                        letra = 'b'
                    elif casilla_inicial == 'q' and casilla_actual == ' ':
                        letra = 'q'
                    elif casilla_inicial == 'k' and casilla_actual == ' ':
                        letra = 'k'

    return letra


#<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3<3
#Funcion del seguimiento de las piezas sin considerar cuando una pieza come a la otra
#def comparar_tableros(tablero_actual, tablero_inicial, letra):
#    cambios = []
#    tablero_resultante = [fila[:] for fila in tablero_inicial]  # Copio el tablero inicial para empezar
#    for i in range(8):
#        for j in range(8):
#            casilla_actual = tablero_actual[i][j]
#            casilla_inicial = tablero_inicial[i][j]
#            if casilla_actual != casilla_inicial:  
#                if casilla_actual == ' ':
#                    cambios.append(f"La pieza en la casilla {chr(65 + j)}{8 - i} se ha movido.")
#                    tablero_resultante[i][j] = casilla_actual # Agrego el espacio en blanco donde antes estaba la pieza ####
#                elif casilla_inicial == ' ':
#                    cambios.append(f"Se movió a la casilla {chr(65 + j)}{8 - i}.")
#                    tablero_resultante[i][j] = letra # Le agrego la letra a la pieza
#
#    return cambios, tablero_resultante

#########################################################################################################
#########################################################################################################
import chess
import chess.svg
from IPython.display import SVG, display

def mostrar_tablero(tablero):
    """Muestra el tablero de ajedrez visualmente."""
    svg_tablero = chess.svg.board(board=tablero, size=500)
    display(SVG(svg_tablero))

def generar_tablero(matriz):
    """Genera un objeto de tablero de ajedrez a partir de una matriz de piezas."""
    fen = ""
    for fila in matriz:
        contador_vacios = 0
        for casilla in fila:
            if casilla == ' ':
                contador_vacios += 1
            else:
                if contador_vacios > 0:
                    fen += str(contador_vacios)
                    contador_vacios = 0
                fen += casilla
        if contador_vacios > 0:
            fen += str(contador_vacios)
        fen += '/'
    fen = fen[:-1]  # Eliminar la última barra diagonal
    return chess.Board(fen=fen)

# Función para determinar de quién es el turno
def obtener_jugador_turno(turno):
    if turno % 2 == 0:
        return "Negras"
    else:
        return "Blancas"
# Funciones utilizadas para cuando una pieza come a la otra ( de aca para abajo)

def comparar_tableros_piezas(tablero_anterior, tablero_actual):
    cambios = []
    for fila in range(len(tablero_anterior)):
        for columna in range(len(tablero_anterior[0])):
            if tablero_anterior[fila][columna] != tablero_actual[fila][columna]:
                cambios.append((fila, columna))
                #print("------------------------------------------------------------------")
                #print("EL TABLERO ANTERIOR ES: ", tablero_anterior)
                #print("EL TABLERO ACTUAL ES: ", tablero_actual)
                #print("------------------------------------------------------------------")
    return cambios

def agregar_letra_en_cambios(tablero, cambios, letra):
    tablero_modificado = [fila[:] for fila in tablero]  # Crear una copia del tablero original
    for cambio in cambios:
        fila, columna = cambio
        tablero_modificado[fila][columna] = letra
    return tablero_modificado

def actualizar_contadores(piezas_verdes_anteriores, piezas_verdes_actuales, tablero_piezas_verdes_anterior, tablero_piezas_verdes_actuales, tablero_resultante, letra):
    tablero_modificado = []  # Inicializamos tablero_modificado como una lista vacía
    if piezas_verdes_actuales < piezas_verdes_anteriores:
        print("Se ha detectado una diferencia en el número de piezas verdes.")
        cambios_p = comparar_tableros_piezas(tablero_piezas_verdes_anterior, tablero_piezas_verdes_actuales)
        #print("El cambio se da en: ",cambios_p)
        #print(" El tablero resultante es: ", tablero_resultante)
        tablero_modificado = agregar_letra_en_cambios(tablero_resultante, cambios_p, letra)
    elif piezas_verdes_actuales == piezas_verdes_anteriores:
        print("Las piezas se mantienen iguales")
    elif piezas_verdes_actuales > piezas_verdes_anteriores:
        print("Error, revisar la jugada.")

    return tablero_modificado


def contar_piezas(tablero):
    contador = 0
    for fila in tablero:
        for elemento in fila:
            if elemento == '-':
                contador += 1
    return contador

#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGD                      FUNCIONES UTILIZADAS PARA STOCKFISH                       GDGDGDGDGD    
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD

import chess
import chess.engine

# Función para convertir la matriz en una cadena de notación FEN (Forsyth-Edwards Notation)
def matriz_a_fen(matriz):
    fen = ""
    for fila in matriz:
        espacios_vacios = 0
        for casilla in fila:
            if casilla == ' ':
                espacios_vacios += 1
            else:
                if espacios_vacios > 0:
                    fen += str(espacios_vacios)
                    espacios_vacios = 0
                fen += casilla
        if espacios_vacios > 0:
            fen += str(espacios_vacios)
        fen += "/"
    # Eliminar la barra diagonal final
    fen = fen[:-1]
    return fen


def fen_a_matriz(fen):
    # Convertir la cadena FEN en una lista de filas
    filas = fen.split('/')
    matriz = []
    for fila in filas:
        fila_matriz = []
        for caracter in fila:
            if caracter.isdigit():
                # Si el caracter es un número, agregar ese número de espacios vacíos
                fila_matriz += [' '] * int(caracter)
            else:
                # Si no, agregar el caracter tal cual
                fila_matriz.append(caracter)
        matriz.append(fila_matriz)
    return matriz


def jugar_como_negras(tablero):
    # Cargar la posición desde la matriz, convertida en fen
    posicion_fen = matriz_a_fen(tablero)
    # Creo el tablero segun la posicion dada
    board = chess.Board(posicion_fen)
    # Cambiar el turno a las negras
    board.turn = chess.BLACK
    #print(board)
    return board

def jugada_Stockfish(board):
    # Iniciar el motor Stockfish
    engine = chess.engine.SimpleEngine.popen_uci("C:\\Users\\Vegan\\OneDrive\\Escritorio\\MATERIAS ACTUALES\\Proyecto Chess Arm\\STOCKFISH\\stockfish\\stockfish-windows-x86-64-sse41-popcnt.exe")#######3

    # Obtener la mejor jugada de Stockfish
    result = engine.play(board, chess.engine.Limit(time=1))

    # Imprimir la mejor jugada
    print("Mejor jugada según Stockfish:", result.move)
    

    # Verificar si result.move es None antes de intentar usarlo
    if result.move is not None:
        # Realizar la jugada
        board.push(result.move)

        # Convertir el tablero actualizado a una cadena FEN
        fen_actualizado = board.fen().split(' ')[0]  # Solo queremos la parte de la posición del FEN

        # Convertir la cadena FEN actualizada a una matriz
        matriz_actualizada = fen_a_matriz(fen_actualizado)

        # Imprimir la matriz actualizada
        # for fila in matriz_actualizada:
        #    print(' '.join(fila))
        return matriz_actualizada
    else:
        # Si result.move es None, no se encontró una jugada válida.
        # En este caso, devolvemos la posición actual del tablero en formato FEN.
        #  ajustar esto según lasnecesidades.
        fen_actualizado = board.fen().split(' ')[0]  # Solo queremos la parte de la posición del FEN
        #IMAGENES
        #image = cv2.imread("Derrota 3.png")
        # Mostrar la imagen
        #cv2.imshow("Imagen", image)
        #cv2.waitKey(20000)  # Esperar 3 segundos (3000 milisegundos)
        #cv2.destroyAllWindows()
        return fen_a_matriz(fen_actualizado)
#tablero =         [['r', ' ', 'b', 'q', 'k', 'b', 'n', 'r'],
#                   ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
#                   ['n', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
#                   [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
#                   [' ', 'P', ' ', ' ', ' ', ' ', ' ', ' '],
#                   [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
#                   ['P', ' ', 'P', 'P', 'P', 'P', 'P', 'P'],
#                   ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']]

#board = jugar_como_negras(tablero)
#matriz_actualizada = jugada_Stockfish(board)

# Imprimir la matriz actualizada
#for fila in matriz_actualizada:
#    print(' '.join(fila))


########################################################################################

def jugada_obtenida_de_Stockfish(tablero_inicial, tablero_actual):
    
    tablero_i= copy.deepcopy(tablero_inicial)
    tablero_a= copy.deepcopy(tablero_actual)
    
    tablero_inicial1 = eliminar_piezas_blancas(tablero_i)
    print(tablero_inicial1)
    tablero_final1 = eliminar_piezas_blancas(tablero_a)
    print(tablero_final1)
    movimiento = []

    # Llama a la primera función y agrega sus resultados a la lista movimiento
    movimiento.extend(comparar_tableros_jugada1(tablero_final1, tablero_inicial1))

    # Llama a la segunda función y agrega sus resultados a la lista movimiento
    movimiento.extend(comparar_tableros_jugada2(tablero_final1, tablero_inicial1))

    #print(movimiento)


    movimiento1 = ''.join(movimiento)
    #print(movimiento1)

    vector_con_espacio = movimiento1[:2] + " " + movimiento1[2:]
    #print(vector_con_espacio)
    #len(vector_con_espacio)
    
    return vector_con_espacio

def contar_piezas_blancas(matriz):
    contador = 0
    for fila in matriz:
        for casilla in fila:
            if casilla.isupper():  # Verificar si la casilla es una letra mayúscula
                contador += 1
    return contador
def verificar_come_o_avanza(matriz_inicial, matriz_ultima, resultado):
    print("asd")
    print(matriz_inicial)
    print("asd")
    print(matriz_ultima)
    contador_blancas_inicial = contar_piezas_blancas(matriz_inicial)
    contador_blancas_ultima = contar_piezas_blancas(matriz_ultima)
    
    if contador_blancas_inicial != contador_blancas_ultima:
        resultado += " 1"  # Concatenar ' 1' a la cadena resultado
    else:
        resultado += " 0"  # Concatenar ' 0' a la cadena resultado
        
    return resultado

# Llamada a la función corregida
#resultado = verificar_come_o_avanza(matriz_inicial, matriz_final, jugada)
#print(resultado)  # Resultado debe ser 'A1 A2 0'
#len(resultado)

#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGD           FUNCIONES UTILIZADAS PARA LA VALIDACION DE MOVIMIENTOS               GDGDGDGDGD    
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD

import chess
import chess.svg
import chess.svg as svg
from IPython.display import SVG, display

def comparar_tableros_jugada1(tablero_actual, tablero_inicial):
    cambios = []
    for i in range(8):
        for j in range(8):
            casilla_inicial = tablero_inicial[i][j]
            casilla_actual = tablero_actual[i][j]
            if casilla_inicial != casilla_actual and casilla_actual == ' ':
                cambios.append(f"{chr(65 + j)}{8 - i}") # Agregar la casilla inicial y la actual

    return cambios  # Invierto la lista para obtener el formato deseado


def comparar_tableros_jugada2(tablero_actual, tablero_inicial):
    cambios2 = []
    for i in range(8):
        for j in range(8):
            casilla_inicial = tablero_inicial[i][j]
            casilla_actual = tablero_actual[i][j]
            if casilla_inicial != casilla_actual and casilla_inicial == ' ':
                cambios2.append(f"{chr(65 + j)}{8 - i}") # Agregar la casilla inicial y la actual
    return cambios2  # Invierto la lista para obtener el formato deseado



#movimiento = []

# Llama a la primera función y agrega sus resultados a la lista movimiento
#movimiento.extend(comparar_tableros_jugada1(tablero1, tablero))

# Llama a la segunda función y agrega sus resultados a la lista movimiento
#movimiento.extend(comparar_tableros_jugada2(tablero1, tablero))

#print(movimiento)

#Reseteo el vector
#movimiento = []
#print(movimiento)


def Validacion_de_piezas_blancas (tablero_actual,tablero_inicial):
    movimiento1 = []

    # Llama a la primera función y agrega sus resultados a la lista movimiento
    movimiento1.extend(comparar_tableros_jugada1(tablero_actual,tablero_inicial))

    # Llama a la segunda función y agrega sus resultados a la lista movimiento
    movimiento1.extend(comparar_tableros_jugada2(tablero_actual,tablero_inicial))

    print(movimiento1)


    print("El movimiento es: ",movimiento1)
    movimiento1 = ''.join(movimiento1)
    # Imprimir la cadena resultante
    print("El movimiento es: ",movimiento1)

    # Convertir la cadena (movimiento) a minúsculas para que me reconosca la jugada y pueda verificar los movimientos 
    movimiento= movimiento1.lower()
    print("El movimiento es: ", movimiento)
    # Cargar la posición desde la matriz convirtiéndola en tipo fen
    posicion_fen = matriz_a_fen(tablero_inicial)
    # Cargo la matriz pasada a fen para crear ahora el tablero
    board = chess.Board(posicion_fen)

    # Supongamos que tienes un movimiento en notación UCI (por ejemplo, 'e2e4')
    move = chess.Move.from_uci(movimiento)
    
    #Reseteo el vector
    movimiento = []
    print(movimiento)
    
    #board.turn = chess.BLACK # Le digo que es el turno de las NEGRAS 
    # Verificar si el movimiento es legal
    if move in board.legal_moves:
        # Si el movimiento es legal, realizar el movimiento
        board.push(move)
        #mensaje = "EL MOVIMIENTO" + str(move) + " ES VALIDO."
        mensaje = "EL MOVIMIENTO ES VALIDO."
        #print("Tablero después del movimiento:")
        #display_board(board)
    else:
        #mensaje = "EL MOVIMIENTO" + str(move) + " NO ES VALIDO."
        mensaje = "EL MOVIMIENTO NO ES VALIDO."
        #print("Tablero después del movimiento:")
        #display_board(board)
        # Cargar la imagen
        #image = cv2.imread("BATMAN1.jpg")
        # Mostrar la imagen
        #cv2.imshow("Imagen", image)
        #cv2.waitKey(5000)  # Esperar 3 segundos (3000 milisegundos)
        #cv2.destroyAllWindows()
    return mensaje

# Esta funcion aun no se utiliza
def cadena_a_tablero(cadena):
    """
    Convierte una cadena de caracteres en una lista de listas representando un tablero de ajedrez.
    """
    filas = cadena.split('\n')
    tablero = []
    for fila in filas:
        fila_tablero = []
        for caracter in fila.split():
            if caracter == '.':
                fila_tablero.append(' ')
            else:
                fila_tablero.append(caracter)
        tablero.append(fila_tablero)
    return tablero


# Esta funcion se utilizo anteriormente, actualmente esta en desuso 
def comparar_tableros(tablero_actual, tablero_inicial, letra):
    cambios = []
    #tablero_resultante = [fila[:] for fila in tablero_inicial]  # Copio el tablero inicial para empezar
    tablero_resultante = copy.deepcopy(tablero_inicial)
    tablero_x = copy.deepcopy(tablero_inicial)
    for i in range(8):
        for j in range(8):
            casilla_actual = tablero_actual[i][j]
            casilla_inicial = tablero_inicial[i][j]
            if casilla_actual != casilla_inicial:  
                if casilla_actual == ' ':
                    cambios.append(f"{chr(65 + j)}{8 - i}")
                    tablero_resultante[i][j] = casilla_actual # Agrego el espacio en blanco donde antes estaba la pieza ####
                elif casilla_inicial == ' ':
                    cambios.append(f"{chr(65 + j)}{8 - i}")
                    tablero_resultante[i][j] = letra # Le agrego la letra a la pieza
                    
    return cambios, tablero_resultante

import copy



def eliminar_piezas_negras(tablero):
    # Iterar sobre cada fila en el tablero
    for fila in range(len(tablero)):
        # Iterar sobre cada columna en la fila actual
        for columna in range(len(tablero[fila])):
            # Verificar si la pieza en la posición actual es negra ('r', 'n', 'b', 'q', 'k', 'p')
            if tablero[fila][columna] in ['r', 'n', 'b', 'q', 'k', 'p']:
                # Reemplazar la pieza negra con un espacio en blanco
                tablero[fila][columna] = ' '
    return tablero


# Eliminar las piezas negras del tablero inicial
#tablero_sin_negras = eliminar_piezas_negras(tablero_inicial)

def eliminar_piezas_blancas(tablero):
    # Iterar sobre cada fila en el tablero
    for fila in range(len(tablero)):
        # Iterar sobre cada columna en la fila actual
        for columna in range(len(tablero[fila])):
            # Verificar si la pieza en la posición actual es negra ('r', 'n', 'b', 'q', 'k', 'p')
            if tablero[fila][columna] in ['R', 'N', 'B', 'Q', 'K', 'P']:
                # Reemplazar la pieza negra con un espacio en blanco
                tablero[fila][columna] = ' '
    return tablero

def Validacion_de_piezas_blancas_al_comer (tablero_actual, tablero_inicial, tablero_inicial1):
    
    movimiento1 = []

    # Llama a la primera función y agrega sus resultados a la lista movimiento
    movimiento1.extend(comparar_tableros_jugada1(tablero_actual,tablero_inicial))

    # Llama a la segunda función y agrega sus resultados a la lista movimiento
    movimiento1.extend(comparar_tableros_jugada2(tablero_actual,tablero_inicial))

    print(movimiento1)
    
    
    #print("El movimiento es: ",movimiento1)
    movimiento1 = ''.join(movimiento1)
    # Imprimir la cadena resultante
    #print("El movimiento es: ",movimiento1)

    # Convertir la cadena (movimiento) a minúsculas para que me reconosca la jugada y pueda verificar los movimientos 
    movimiento= movimiento1.lower()
    print("El movimiento es: ", movimiento)
    # Cargar la posición desde la matriz convirtiéndola en tipo fen
    posicion_fen = matriz_a_fen(tablero_inicial1)
    # Cargo la matriz pasada a fen para crear ahora el tablero
    board = chess.Board(posicion_fen)



    # Supongamos que tienes un movimiento en notación UCI (por ejemplo, 'e2e4')
    move = chess.Move.from_uci(movimiento)
    
    #Reseteo el vector
    movimiento1 = []
    print(movimiento1) 
    
    #board.turn = chess.BLACK # Le digo que es el turno de las NEGRAS 
    # Verificar si el movimiento es legal
    if move in board.legal_moves:
        # Si el movimiento es legal, realizar el movimiento
        board.push(move)
        #print("EL MOVIMIENTO" + str(move) + " ES VALIDO.")
        mensaje = "EL MOVIMIENTO ES VALIDO."
        #print("Tablero después del movimiento:")
        #display_board(board)
    else:
        #print("EL MOVIMIENTO" + str(move) + " NO ES VALIDO.")
        mensaje = "EL MOVIMIENTO NO ES VALIDO."
        #print("Tablero después del movimiento:")
        #display_board(board)
        # Cargar la imagen
        #image = cv2.imread("BATMAN1.jpg")
        # Mostrar la imagen
        #cv2.imshow("Imagen", image)
        #cv2.waitKey(5000)  # Esperar 3 segundos (3000 milisegundos)
        #cv2.destroyAllWindows()
    return mensaje


#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGD               FUNCIONES PARA LA CORONACION DE PIEZAS BLANCAS                   GDGDGDGDGD    
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD

def coronar_peon(tablero, color):
    # Determinar la fila de coronación según el color del peón
    fila_coronacion = 7 if color == 'negro' else 0  #Si el color es 'negro', la fila de coronación será la fila 7 (contando desde cero); de lo contrario, si el color es 'blanco', la fila de coronación será la fila 0.
    # Determinar la pieza a coronar según el color del peón
    pieza_coronacion = 'Q' if color == 'blanco' else 'q'  # Si el color es blanco, coronar como reina (Q); si es negro, coronar como reina (q)
    
    
    # Coronar los peones en la fila de coronación
    for columna in range(len(tablero[fila_coronacion])):  # Iterar sobre las columnas de la fila de coronación
        if tablero[fila_coronacion][columna] == 'P' and color == 'blanco':  # Si es un peón blanco
            tablero[fila_coronacion][columna] = pieza_coronacion  # Coronar como reina
        elif tablero[fila_coronacion][columna] == 'p' and color == 'negro':  # Si es un peón negro
            tablero[fila_coronacion][columna] = pieza_coronacion  # Coronar como reina

    return tablero



# Simular coronación de los peones
#tablero_actualizado = coronar_peon(tablero, 'blanco')  # Coronar peones blancos
#tablero_actualizado = coronar_peon(tablero_actualizado, 'negro')  # Coronar peones negros

# Imprimir el tablero actualizado
#for fila in tablero_actualizado:
#    print(fila)   

#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGD                FUNCIONES PARA LA CORONACION DE PIEZAS NEGRAS                   GDGDGDGDGD    
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD

def coronacion_piezas_negras(matriz_inicial, matriz_actual):
    # Verificar si hay peones negros en la fila 6 de la matriz inicial
    if 'p' in matriz_inicial[6]:
        # Variable para verificar si se debe coronar alguna pieza
        coronar_pieza = False
        
        # Iterar sobre las columnas de la matriz
        for columna in range(len(matriz_actual[6])):
            # Verificar si en la matriz inicial había un peón y ahora hay una nueva pieza en la misma posición
            if matriz_inicial[6][columna] == 'p' and matriz_actual[6][columna] != 'p':
                coronar_pieza = True
                # No necesitamos seguir buscando peones en esta fila, rompemos el bucle
                break
                
        # Si se debe coronar alguna pieza, procedemos a comparar las últimas filas
        if coronar_pieza:
            # Obtener la última fila de ambas matrices
            ultima_fila_inicial = matriz_inicial[-1]
            ultima_fila_actual = matriz_actual[-1] 
            
            # Verificar si hay una diferencia entre las últimas filas
            for i in range(len(ultima_fila_inicial)):
                if ultima_fila_inicial[i] != ultima_fila_actual[i]:
                    # Colocar 'q' en la posición donde hay diferencia
                    matriz_actual[-1][i] = 'q'

    return matriz_actual

#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGD                          FUNCIONES PARA LOS ENROQUES                           GDGDGDGDGD    
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD

import copy
#numero = 0
def verificar_enroque_corto(matriz_inicial, matriz_actual):
    # Verificar si el rey blanco está en su posición inicial
    if matriz_inicial[7][4] == 'K' and matriz_inicial[7][7] == 'R' and matriz_inicial[7][5] == ' ' and matriz_inicial[7][6] == ' ':#####
        if matriz_actual[7][4] == ' ' and matriz_actual[7][7] == ' ':
            numero = 1
            tablero_enroque = copy.deepcopy(matriz_actual)
            tablero_enroque[7][5] = 'R'
            tablero_enroque[7][6] = 'K'

            return tablero_enroque, 1
    return matriz_actual, 0

#matriz, numero = verificar_enroque_corto(matriz_inicial, matriz_actual)

def verificar_enroque_largo(matriz_inicial, matriz_actual):
    # Verificar si el rey blanco está en su posición inicial
    if matriz_inicial[7][4] == 'K' and matriz_inicial[7][0] == 'R' and matriz_inicial[7][1] == ' ' and matriz_inicial[7][2] == ' ' and matriz_inicial[7][3] == ' ':#####
        print("asd")
        if matriz_actual[7][0] == ' ' and matriz_actual[7][4] == ' ':
            print("asd1")
            numero = 1
            tablero_enroque = copy.deepcopy(matriz_actual)
            tablero_enroque[7][3] = 'R'
            tablero_enroque[7][2] = 'K'

            return tablero_enroque, 1
    return matriz_actual, 0

#matriz, numero = verificar_enroque_largo(matriz_inicial, matriz_actual)


###################################################################################################
########################## FUNCIONES PARA LOS ENROQUES DE STOCKFISH ###############################

import copy
#numero = 0
def verificar_enroque_corto1(matriz_inicial, matriz_actual):
    # Verificar si el rey blanco está en su posición inicial
    if matriz_inicial[0][4] == 'k' and matriz_inicial[0][7] == 'r' and matriz_inicial[0][5] == ' ' and matriz_inicial[0][6] == ' ':#####
        if matriz_actual[0][4] == ' ' and matriz_actual[0][7] == ' ':
            numero = 1
            tablero_enroque = copy.deepcopy(matriz_actual)
            tablero_enroque[0][5] = 'r'
            tablero_enroque[0][6] = 'k'

            return tablero_enroque, 1
    return matriz_actual, 0

#matriz, numero = verificar_enroque_corto1(matriz_inicial, matriz_actual)

def verificar_enroque_largo1(matriz_inicial, matriz_actual):
    # Verificar si el rey blanco está en su posición inicial
    if matriz_inicial[0][4] == 'k' and matriz_inicial[0][0] == 'r' and matriz_inicial[0][1] == ' ' and matriz_inicial[0][2] == ' ' and matriz_inicial[0][3] == ' ':#####
        print("asd")
        if matriz_actual[0][0] == ' ' and matriz_actual[0][4] == ' ':
            print("asd1")
            numero = 1
            tablero_enroque = copy.deepcopy(matriz_actual)
            tablero_enroque[0][3] = 'r'
            tablero_enroque[0][2] = 'k'

            return tablero_enroque, 1
    return matriz_actual, 0

#matriz, numero = verificar_enroque_largo1(matriz_inicial, matriz_actual)


#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGD                      FUNCIONES PARA LA CAPTURA AL PASO                         GDGDGDGDGD    
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD

#                                     --------------------

#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGD    FUNCIONES PARA GENERAR LA VISUALIZACION DEL TABLERO EN UNA VENTANA APARTE   GDGDGDGDGD    
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
# Ruta de la carpeta donde están las imágenes de las piezas para casillas blancas y negras
#IMG_FOLDER_BLANCA = r'C:\\Users\\Vegan\\OneDrive\\Escritorio\\fotos de las piezas originales\\fotos de las piezas para visualizar\\fotos de las piezas\\Blancas'
#IMG_FOLDER_NEGRA = r'C:\\Users\\Vegan\\OneDrive\\Escritorio\\fotos de las piezas originales\\fotos de las piezas para visualizar\\fotos de las piezas\\Negras'
import cv2
import numpy as np
import time

# Rutas de las carpetas donde están las imágenes de las piezas para casillas blancas y negras
IMG_FOLDER_BLANCA = r'C:\Users\Vegan\OneDrive\Escritorio\fotos de las piezas originales\Blancas - copia'
IMG_FOLDER_NEGRA = r'C:\Users\Vegan\OneDrive\Escritorio\fotos de las piezas originales\Negras - copia'

# Función para cargar las imágenes de las piezas desde las carpetas especificadas
def load_piece_images():
    piece_images = {}
    pieces = ['p1', 'q1', 'k1', 'b1', 'n1', 'r1','P', 'Q', 'K', 'B', 'N', 'R']
    
    # Cargar imágenes para casillas blancas
    for piece in pieces:
        img_path = f"{IMG_FOLDER_BLANCA}\\{piece}.png"
        img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        if img is not None:
            # Convertir a 3 canales si es RGBA
            if img.shape[2] == 4:
                img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
            piece_images[f'blanca_{piece}'] = cv2.resize(img, (55, 55))  # Ajustar tamaño si es necesario
        else:
            print(f"Imagen no encontrada para la pieza: {piece} en color blanca")
            piece_images[f'blanca_{piece}'] = None

    # Cargar imágenes para casillas negras
    for piece in pieces:
        img_path = f"{IMG_FOLDER_NEGRA}\\{piece}.png"
        img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        if img is not None:
            # Convertir a 3 canales si es RGBA
            if img.shape[2] == 4:
                img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
            piece_images[f'negra_{piece}'] = cv2.resize(img, (55, 55))  # Ajustar tamaño si es necesario
        else:
            print(f"Imagen no encontrada para la pieza: {piece} en color negra")
            piece_images[f'negra_{piece}'] = None

    return piece_images


# Función para mostrar el tablero con las imágenes de las piezas y mensajes
def mostrar_tableroasd(tablero, piece_images, mensajes, otro_mensaje, TURNOS):
    cuadro_size = 55
    # Si se quiere un fondo completamente negro descomentar
    #tablero_imagen = np.zeros((12*cuadro_size, 15*cuadro_size, 3), dtype=np.uint8)  # Ajustar tamaño para incluir los bordes

    # Para cargar una imagen de fondo
    ########################################################################################
    # Cargar la imagen de fondo
    background_image = cv2.imread('Nikola-Tesla-fondotablero.png')
    # Redimensionar la imagen de fondo para que coincida con el tamaño del tablero
    background_image = cv2.resize(background_image, (15*cuadro_size, 12*cuadro_size))
    
    # Crear la imagen del tablero usando la imagen de fondo
    tablero_imagen = background_image.copy()
    ########################################################################################
    
    # Colores para las casillas
    color_blanco = (241, 241, 241)  # Lila
    color_negro = (187, 118, 131)       # Rojo

    # Colocar números en el borde izquierdo y derecho
    for i in range(8):
        cv2.putText(tablero_imagen, str(8 - i), (cuadro_size//4, cuadro_size*(i+1) + cuadro_size//2),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 1)
        cv2.putText(tablero_imagen, str(8 - i), (9*cuadro_size + cuadro_size//4, cuadro_size*(i+1) + cuadro_size//2),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 1)

    # Colocar letras en la parte superior e inferior
    letras = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    for j in range(8):
        cv2.putText(tablero_imagen, letras[j], (cuadro_size*(j+1) + cuadro_size//4, cuadro_size//2),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 1)
        cv2.putText(tablero_imagen, letras[j], (cuadro_size*(j+1) + cuadro_size//4, 9*cuadro_size + cuadro_size//2),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 1)

    # Dibujar el tablero y las piezas
    for fila in range(8):
        for columna in range(8):
            if (fila + columna) % 2 == 0:
                color = color_blanco
            else:
                color = color_negro

            y1 = (fila + 1) * cuadro_size
            y2 = (fila + 2) * cuadro_size
            x1 = (columna + 1) * cuadro_size
            x2 = (columna + 2) * cuadro_size

            tablero_imagen[y1:y2, x1:x2] = color

            if tablero[fila][columna] != ' ':
                pieza = tablero[fila][columna]

                if color == color_blanco:
                    key_blancas = f"blanca_{pieza}"
                    key_blancas_sufijo = f"blanca_{pieza}1"
                else:
                    key_negras = f"negra_{pieza}"
                    key_negras_sufijo = f"negra_{pieza}1"

                # Verificar si existe la imagen con el sufijo y si no, usar la imagen base
                if color == color_blanco and key_blancas_sufijo in piece_images and piece_images[key_blancas_sufijo] is not None:
                    tablero_imagen[y1:y2, x1:x2] = piece_images[key_blancas_sufijo]
                elif color == color_blanco and key_blancas in piece_images and piece_images[key_blancas] is not None:
                    tablero_imagen[y1:y2, x1:x2] = piece_images[key_blancas]
                elif color == color_negro and key_negras_sufijo in piece_images and piece_images[key_negras_sufijo] is not None:
                    tablero_imagen[y1:y2, x1:x2] = piece_images[key_negras_sufijo]
                elif color == color_negro and key_negras in piece_images and piece_images[key_negras] is not None:
                    tablero_imagen[y1:y2, x1:x2] = piece_images[key_negras]
                    

    # Coloco el titulo
    titulo = "TURNO:"
    cv2.putText(tablero_imagen, titulo, (11*cuadro_size, 6*cuadro_size),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 1)  
    # Mostrar otro mensaje en la parte inferior del tablero
    cv2.putText(tablero_imagen, TURNOS, (11*cuadro_size, 7*cuadro_size),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    # Mostrar mensajes al costado del tablero
    #max_mensajes = 4  # Máximo número de mensajes a mostrar
    #if len(mensajes) > max_mensajes:
    #    mensajes_a_mostrar = mensajes[-max_mensajes:]  # Mostrar los últimos max_mensajes mensajes
    #else:
    #    mensajes_a_mostrar = mensajes

    #for i, mensaje in enumerate(mensajes_a_mostrar, start=1):
        #mensaje_con_numero = f"{i}: {mensaje}"  # Construir el mensaje con el número
     #   mensaje_con_numero = mensaje  # Solo el mensaje sin el número
      #  cv2.putText(tablero_imagen, mensaje_con_numero, (11*cuadro_size , cuadro_size*(i+1)),
       #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    # Coloco el titulo
    titulo = "JUGADAS:"
    cv2.putText(tablero_imagen, titulo, (11*cuadro_size , cuadro_size),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 1)
    
    cv2.putText(tablero_imagen, mensajes, (11*cuadro_size, 2*cuadro_size),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        # Mostrar otro mensaje en la parte inferior del tablero

        
    # Coloco el titulo
    titulo = "Validacion de Jugadas:"
    cv2.putText(tablero_imagen, titulo, (cuadro_size,11*cuadro_size),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)
    # Mostrar otro mensaje en la parte inferior del tablero
    cv2.putText(tablero_imagen, otro_mensaje, (cuadro_size, 11*cuadro_size + cuadro_size//2),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    
    cv2.imshow('Tablero de Ajedrez', tablero_imagen)
    cv2.moveWindow('Tablero de Ajedrez', 1, 1)




#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGD                      FUNCION PARA PREDER LED CON ARDUINO                       GDGDGDGDGD    
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD

import serial
import time

# Configurar el puerto serial (ajusta el puerto según tu configuración)
#arduino_port = 'COM7'  # Puerto COM al que está conectado tu Arduino en Windows

#def encender_led(valor):
    # Inicializar la comunicación serial
#    ser = serial.Serial(arduino_port, 9600)
#    time.sleep(2)  # Esperar a que la conexión serial se estabilice

    # Enviar un mensaje a Arduino
#    mensaje = valor
#    ser.write(mensaje.encode())

    # Cerrar la conexión serial
#    ser.close()

def verificar_ultimo_elemento(vector):
    # Verificamos si el último elemento es '0' o '1'
    if vector[-1] == '0':
        print("El último elemento es 0")
        encender_led('0')
    elif vector[-1] == '1':
        print("El último elemento es 1")
        encender_led('1')
    else:
        print("El último elemento no es ni 0 ni 1")


#verificar_ultimo_elemento(resultado)





#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGD              FUNCIONES PARA OBTENER LAS JUGADAS Y VISUALIZARLAS                GDGDGDGDGD    
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
import copy
def eliminar_piezas_negras(tablero):
    # Iterar sobre cada fila en el tablero
    for fila in range(len(tablero)):
        # Iterar sobre cada columna en la fila actual
        for columna in range(len(tablero[fila])):
            # Verificar si la pieza en la posición actual es negra ('r', 'n', 'b', 'q', 'k', 'p')
            if tablero[fila][columna] in ['r', 'n', 'b', 'q', 'k', 'p']:
                # Reemplazar la pieza negra con un espacio en blanco
                tablero[fila][columna] = ' '
    return tablero

def comparar_tableros_blancas(tablero_actual, tablero_inicial):
    cambios1 = ""
    tablero_a = copy.deepcopy(tablero_actual)
    tablero_i = copy.deepcopy(tablero_inicial)
    tablero_ac = eliminar_piezas_negras(tablero_a)
    tablero_in = eliminar_piezas_negras(tablero_i)

    for i in range(8):
        for j in range(8):
            if tablero_ac[i][j] != tablero_in[i][j]:
                if tablero_ac[i][j] == " ":
                    fila_origen = 8 - i
                    columna_origen = chr(j + 97)
                    cambios1 += f"{columna_origen}{fila_origen} "
                    cambios1 = cambios1.upper()

    return cambios1

def comparar_tableros2_blancas(tablero_actual, tablero_inicial):
    cambios2 = ""
    tablero_a = copy.deepcopy(tablero_actual)
    tablero_i = copy.deepcopy(tablero_inicial)
    tablero_ac = eliminar_piezas_negras(tablero_a)
    tablero_in = eliminar_piezas_negras(tablero_i)
    for i in range(8):
        for j in range(8):
            if tablero_ac[i][j] != tablero_in[i][j]:
                if tablero_in[i][j] == " ":
                    fila_origen = 8 - i
                    columna_origen = chr(j + 97)
                    cambios2 += f"{columna_origen}{fila_origen} "
                    cambios2 = cambios2.upper()
    return cambios2
def Jugada_Blancas_Visualizar(cambios1, cambios2):
    
    cambios3 = cambios1 + cambios2

    return cambios3

import copy
def eliminar_piezas_blancas(tablero):
    # Iterar sobre cada fila en el tablero
    for fila in range(len(tablero)):
        # Iterar sobre cada columna en la fila actual
        for columna in range(len(tablero[fila])):
            # Verificar si la pieza en la posición actual es negra ('r', 'n', 'b', 'q', 'k', 'p')
            if tablero[fila][columna] in ['R', 'N', 'B', 'Q', 'K', 'P']:
                # Reemplazar la pieza negra con un espacio en blanco
                tablero[fila][columna] = ' '
    return tablero

def comparar_tableros_negras(tablero_actual, tablero_inicial):
    cambios1 = ""
    tablero_a = copy.deepcopy(tablero_actual)
    tablero_i = copy.deepcopy(tablero_inicial)
    tablero_ac = eliminar_piezas_blancas(tablero_a)
    tablero_in = eliminar_piezas_blancas(tablero_i)

    for i in range(8):
        for j in range(8):
            if tablero_ac[i][j] != tablero_in[i][j]:
                if tablero_ac[i][j] == " ":
                    fila_origen = 8 - i
                    columna_origen = chr(j + 97)
                    cambios1 += f"{columna_origen}{fila_origen} "
                    cambios1 = cambios1.upper()
    return cambios1

def comparar_tableros2_negras(tablero_actual, tablero_inicial):
    cambios2 = ""
    tablero_a = copy.deepcopy(tablero_actual)
    tablero_i = copy.deepcopy(tablero_inicial)
    tablero_ac = eliminar_piezas_blancas(tablero_a)
    tablero_in = eliminar_piezas_blancas(tablero_i)
    for i in range(8):
        for j in range(8):
            if tablero_ac[i][j] != tablero_in[i][j]:
                if tablero_in[i][j] == " ":
                    fila_origen = 8 - i
                    columna_origen = chr(j + 97)
                    cambios2 += f"{columna_origen}{fila_origen} "
                    cambios2 = cambios2.upper()
    return cambios2
def Jugada_Negras_Visualizar(cambios1, cambios2):
    
    cambios3 = cambios1 + cambios2

    return cambios3





<span style="color:pink; font-size:32px; font-family:'Times New Roman'">**PUERTO SERIE**</span>

<!-- 

color: cambia el color del texto 
font-size: establece el tamaño de la letra en píxeles.
font-family:'Times New Roman' cambia el tipo de letra a "Times New Roman".
** se utiliza para hacer que el texto del título esté en negrita

-->

In [34]:
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGD                  FUNCIONES PARA PASARLE A ARDUINO LAS JUGADAS                  GDGDGDGDGD    
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
#GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
# Configura el puerto serie
arduino_port = 'COM5'  # Cambia esto al puerto correcto en tu sistema
baud_rate = 9600       # Velocidad de comunicación con Arduino


arduino = serial.Serial(arduino_port, baud_rate,timeout=1)

def convertir_a_vector(coordenadas_str):
    # Dividir el string en tres partes de longitud igual
    partes = [coordenadas_str[i:i+3] for i in range(0, len(coordenadas_str), 3)]
    
    # Remover cualquier espacio en blanco alrededor de cada parte
    partes = [parte.strip() for parte in partes]
    
    return partes




def enviar_coordenadas_a_arduino(coordenadas1):
    print("entro a la funcion")
    print(coordenadas1)
    
    coordenadas = convertir_a_vector(coordenadas1)
    print(coordenadas)  # Output: ['D7', 'D6', '0']
    
    if len(coordenadas) == 3:  # Ahora coordenadas tiene 3 elementos (inicial, final, es_comer)
        coord_inicial = coordenadas[0].replace('J', 'D').replace('K', 'A')
        coord_final = coordenadas[1].replace('J', 'D').replace('K', 'A')
        es_comer = coordenadas[2]
        mensaje = f"{coord_inicial} {coord_final} {es_comer}\n"
        arduino.write(mensaje.encode())
        print(f"Enviando coordenadas: {mensaje}")
    else:
        print("Se necesitan exactamente dos coordenadas válidas para enviar.")

<span style="color:red; font-size:32px; font-family:'Times New Roman'">**Bucle principal**</span>

<!-- 

color: cambia el color del texto 
font-size: establece el tamaño de la letra en píxeles.
font-family:'Times New Roman' cambia el tipo de letra a "Times New Roman".
** se utiliza para hacer que el texto del título esté en negrita

-->

In [1]:
# ANTES DE EJECUTAR ESTA CELDA PROCURAR TENER EL CIRCUITO ENERGIZADO CON LA FUENTE
import cv2
import serial
import time


# Inicializo mi contador
count = 0

# Inicializo la cámara
cap = cv2.VideoCapture(1)

# Variable para controlar si se ha realizado el proceso
proceso_realizado = False

while True:
    ret, frame = cap.read()

    # Muestro la imagen en una ventana
    cv2.imshow('Reconocer Tablero', frame)

    # Manejo de eventos
    if arduino.in_waiting > 0:
        #line = arduino.readline().decode('utf-8').strip()
        line = arduino.readline().decode('utf-8', errors='ignore').strip()
        if line == 'CAPTURE':
            # Capturo la imagen si se recibe la señal 'CAPTURE'
            filename = f'captura_{count}.png'
            cv2.imwrite(filename, frame)
            print(f"Captura guardada como {filename}")
            count += 1  # Incrementa el contador para la siguiente imagen
            
            if not proceso_realizado:
                #x_coords, y_coords, imagen1, nombre_variable = deteccion(127,58,87,149,255,255, frame,  nombre_variable = "coordenadas_esquinas_caja")#####
                #esquinas = todas_las_coordenadas(x_coords, y_coords)
                #casillas_y_sus_puntos_medios(esquinas, imagen1)
                # Armado del tablero 
                coordenadas_esquinas = detector_por_color_esquinas(Am1, Am2, Am3, Am4, Am5, Am6, frame, nombre_variable="coordenadas_esquinas")
                MatrizTraspuesta = Armado_de_nodos_de_las_casillas(coordenadas_esquinas, frame)
                
                # Marcar que el proceso ha sido realizado
                proceso_realizado = True

    # Manejo de eventos de teclado
    key = cv2.waitKey(1)
    if key == 27:  # Tecla ESC para salir del bucle
        break

    # Salir del bucle si el proceso ha sido realizado
    if proceso_realizado:
        break

# Liberar la cámara y cerrar todas las ventanas
cap.release()
cv2.destroyAllWindows()


#IMAGENES
#image0 = cv2.imread("Acomodar 1.png")
# Mostrar la imagen
#cv2.imshow("Imagen", image0)
#cv2.waitKey(10000)# Esperar 3 segundos (3000 milisegundos)
#cv2.destroyAllWindows()
#
# Inicializo el contador de turnos
turno = 0  # Comienza con las piezas blancas

# Utilizado para saltar la validacion de movimiento en los enroque
numero = 0

# Inicializo mi contador
count = 0

#mensajes = []
mensajes = ""
mensajes2 = ""

# Vector para pasarle a arduino los movimientos 
movimiento = []

print("Las coordenadas que conforman la esquina son: ",coordenadas_esquinas)
# Lo conviértelo a un array de NumPy de tipo entero
puntos_cuadrado = np.array(coordenadas_esquinas, dtype=np.int32)

# Cambio la forma a la requerida por cv2.fillPoly
puntos_cuadrado = puntos_cuadrado.reshape((-1, 1, 2))



# Inicializo la cámara
cap = cv2.VideoCapture(1)

# Variable para controlar la captura única
captura_realizada = False

count = 0

# Cargo las imágenes de las piezas para visualizarlas en el tablero
piece_images = load_piece_images()

while True:
    ret, frame = cap.read()
    # Creo una máscara con las mismas dimensiones que el frame
    mascara = np.zeros_like(frame)

    # Relleno el cuadrado de la máscara con blanco-negro
    cv2.fillPoly(mascara, [puntos_cuadrado], (255,255,255))

    # Aplico la máscara al frame
    frame = cv2.bitwise_and(frame, mascara)
    
    # Muestro la imagen en una ventana
    cv2.imshow('PARTIDA', frame)
    # Manejo de eventos
    if arduino.in_waiting > 0:
        line = arduino.readline().decode('utf-8').strip()
        if line == 'CAPTURE' and not captura_realizada:
            # Capturo la imagen si se recibe la señal 'CAPTURE'
            filename = f'captura_{count}.png'
            cv2.imwrite(filename, frame)
            print(f"Captura guardada como {filename}")

        ##############################################################################################

            # Detecto las nuevas posiciones de las piezas (en este caso lo uso para las posiciones iniciales)
            coordenadas_piezas_nuevas = detectar_nuevas_posiciones(coordenadas_esquinas, frame)

            # Etiquetado de la posicion inicial
            tablero_inicial = Etiquetado_de_piezas_en_cada_casilla(coordenadas_piezas_nuevas, MatrizTraspuesta)
            #---------------------------------------------------------------------------------------------------
            coordenadas_verde = detector_por_color(Ve1, Ve2, Ve3, Ve4, Ve5, Ve6, frame, nombre_variable = "coordenadas_piezas_verdes")
            tablero_verde = etiquetar_piezas_actuales(coordenadas_verde, MatrizTraspuesta)   
            
            cantidad_piezas_verdes_anterior = contar_piezas(tablero_verde)

            coordenadas_azul = detector_por_color(Az1, Az2, Az3, Az4, Az5, Az6, frame, nombre_variable = "coordenadas_piezas_azules")
            tablero_azul = etiquetar_piezas_actuales(coordenadas_azul, MatrizTraspuesta)  
            
            cantidad_piezas_azules_actual = contar_piezas(tablero_azul)
            #print("La cantidad de piezas azules actual en el tablero es:", cantidad_piezas_azules_actual)  
        
            tablero = generar_tablero(tablero_inicial)
            mostrar_tablero(tablero)  
            mensajes = "Esperando Jugadas"
            otro_mensaje = "Verificacion de Validacion"
            TURNOS = "PIEZAS BLANCAS"
            mostrar_tableroasd(tablero_inicial, piece_images, mensajes, otro_mensaje, TURNOS)
            mensajes = ""
            # Marcar que la captura ha sido realizada
            captura_realizada = True
        

            # Inicializo los tableros con las piezas verdes y azules por separado
            tablero_piezas_verdes_anterior = tablero_verde    # Al tablero verde lo obtengo del bucle anterior
            tablero_piezas_azules_anterior = tablero_azul     # Al tablero azul lo obtengo del bucle anterior

            # Inicializo la cantidad de piezas verdes y azules obtenidas del bucle anterior
            cantidad_piezas_verdes_anterior = contar_piezas(tablero_verde)
            cantidad_piezas_azules_anterior = contar_piezas(tablero_azul)
            

        
        elif line == 'CAPTURE' and captura_realizada:

            # Incrementar el contador de turnos después de cada turno
            turno += 1
        
            jugador_turno = obtener_jugador_turno(turno) 

            # Piezas Verdes = Piezas Blancas
            if jugador_turno == "Blancas":
                
                
                print("-----------------------------------------------------------------------------------")
                print("---------------------------Turno de las piezas Blancas-----------------------------")
        
                coordenadas_azul = detector_por_color2(Az1, Az2, Az3, Az4, Az5, Az6, frame, nombre_variable = "coordenadas_piezas_azules")
                tablero_azul = etiquetar_piezas_actuales(coordenadas_azul, MatrizTraspuesta)  
                cantidad_piezas_azules_actual = contar_piezas(tablero_azul)
                #print("La cantidad de piezas azules actual en el tablero es:", cantidad_piezas_azules_actual)
            
                coordenadas_verde = detector_por_color2(Ve1, Ve2, Ve3, Ve4, Ve5, Ve6, frame, nombre_variable = "coordenadas_piezas_verdes")
                tablero_verde = etiquetar_piezas_actuales(coordenadas_verde, MatrizTraspuesta)     
                cantidad_piezas_verdes_actual = contar_piezas(tablero_verde)
                #print("La cantidad de piezas verdes actual en el tablero es:", cantidad_piezas_verdes_actual)        
            
                # El ser iguales la cantidades de las piezas, implica que ninguna pieza fue comida         
                if cantidad_piezas_azules_anterior == cantidad_piezas_azules_actual: 
                    # Detecto las nuevas posiciones de ambas piezas en conjunto (verdes y azules)
                    coordenadas_piezas = detectar_nuevas_posiciones(coordenadas_esquinas, frame)
                    tablero_actual = etiquetar_piezas_actuales(coordenadas_piezas, MatrizTraspuesta)    
                
                    # Obtengo la letra que mapeo su posicion
                    letra = letras(tablero_actual, tablero_inicial,turno)
                

                    # Obtengo mi matriz con la jugada realizada
                    cambios, tablero_resultante= comparar_tableros(tablero_actual, tablero_inicial, letra)
                     
                
                    # VERIFICACION DE LA CORONACION DEL PEON BLANCO
                    tablero_resultante1 = coronar_peon(tablero_resultante, 'blanco')  # Coronar peones blancos
                
                    # VERIFICACION DEL ENROQUE CORTO
                    tablero_resultante3, numero = verificar_enroque_corto(tablero_inicial, tablero_resultante1)##
                    #print(numero)
                
                    # VERIFICACION DEL ENROQUE LARGO
                    if numero != 1:
                        tablero_resultante3, numero = verificar_enroque_largo(tablero_inicial, tablero_resultante3)
                    #print(numero)
                
                    print("El tablero resultante essss:") #Tablero inical pero obtenido de otra matriz
                    for fila in tablero_resultante3 :
                        print(fila)   
                
                    # Genero una visualizacion mas realista de las jugadas
                    tablero1= generar_tablero(tablero_resultante3)
                    mostrar_tablero(tablero1)
                

    

                    if numero == 0:  # Si no se produjo el enroque corto 
                        # VALIDACION DE MOVIMIENTOS 
                        mensaje = Validacion_de_piezas_blancas(tablero_resultante3 ,tablero_inicial)####
                    print(mensaje)
                    #print("El numero es: ",numero)   
                
                    cambios1 = comparar_tableros_blancas(tablero_resultante3, tablero_inicial)
                    print("Cambios detectados:")
                    print(cambios)
                    cambios2 = comparar_tableros2_blancas(tablero_resultante3, tablero_inicial)
                    print("Cambios detectados:")
                    print(cambios2)
                    cambios3 = Jugada_Blancas_Visualizar(cambios1, cambios2)
                    print(cambios3)

                    mensajes = cambios3
                
                    #mensajes.append("Negras: A8 A6")
                    otro_mensaje = mensaje
                    TURNOS = "PIEZAS BLANCAS"
                    mostrar_tableroasd(tablero_resultante3, piece_images, mensajes, otro_mensaje, TURNOS)
                    mensajes = ""
                    # Reseteo "numero" al valor original
                    numero = 0
                    #print("El numero es: ",numero)
                
                
                
                    #STOCKFISH:
                    print("las Piezas negras responden con: ")
                    # tablero_inicial 
                    board = jugar_como_negras(tablero_resultante3)
                    matriz_actualizada = jugada_Stockfish(board)
                
                    #GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
                    # ESPERO 3 SEGUNDOS ANTES DE MOSTRAR LA JUGADA DE STOCKFISH pero sin detener la camara
                    # Guardo el tiempo de inicio del delay
                    start_time = time.time()
        
                    # Esperar 3 segundos antes de mostrar el siguiente tablero
                    while time.time() - start_time < 3:
                        #
                        # Mientras se espera, se puede realizar algún procesamiento ligero si es necesario
                        # por ejemplo, leer la cámara o manejar otros eventos
                        ret, frame = cap.read()
                        frame = cv2.bitwise_and(frame, mascara)
                        cv2.imshow('PARTIDA', frame)
                        cv2.waitKey(1)  
                    

                    #GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD                            
                    # Parametros de la jugada para pasarlo a arduino
                    jugada = jugada_obtenida_de_Stockfish(tablero_inicial, matriz_actualizada)
                    print("Stockfish jugo con: ",jugada)

                
                    cambios1 = comparar_tableros_negras(matriz_actualizada, tablero_inicial)
                    print("Cambios detectados:")
                    print(cambios1)
                    cambios2 = comparar_tableros2_negras(matriz_actualizada, tablero_inicial)
                    print("Cambios detectados:")
                    print(cambios2)
                    cambios3 = Jugada_Negras_Visualizar(cambios1, cambios2)
                    print(cambios3)
                
                    mensajes = cambios3
                    #mensajes.append("tuple(jugada)")  # Convertimos jugada a tupla y la añadimos a mensajes
                    otro_mensaje = "EL MOVIMIENTO ES VALIDO."
                    TURNOS = "PIEZAS NEGRAS"
                    mostrar_tableroasd(matriz_actualizada, piece_images,  mensajes, otro_mensaje, TURNOS)                
                    mensajes = ""
                    
                    # Le agrego las ultimas dos variables ( 0) si solo avanza o ( 1) si ademas de avanzar come
                    resultado = verificar_come_o_avanza(tablero_inicial, matriz_actualizada, jugada)
                    print(resultado)  # Resultado
                    
                
                    #verificar_ultimo_elemento(resultado)
                    enviar_coordenadas_a_arduino(resultado)
                
                       
                
                    # Genero una visualizacion mas realista de las jugadas
                    tablero = generar_tablero(matriz_actualizada)
                    mostrar_tablero(tablero)

                
                    # Reinicio
                    jugada = []
                    # Actualizo el tablero inicial con el tablero resultante
                    tablero_inicial = tablero_resultante3

                    # Vacio la variable letra
                    letra = ' '
                
                    #print("-----------------------------------------------------------------------------------")  
                    #print("El tablero anterior de las piezas azules es: ")
                    #print(tablero_piezas_azules_anterior)  
                    #actualizo la cantidad de piezas azules:
                    cantidad_piezas_azules_anterior = cantidad_piezas_azules_actual
                    tablero_piezas_azules_anterior = tablero_azul
                    #print("El tablero actual de las piezas azules es: ")
                    #print(tablero_piezas_azules_anterior)
                    #print("-----------------------------------------------------------------------------------")  
                    #print("El tablero anterior de las piezas verdes es: ")
                    #print(tablero_piezas_verdes_anterior)
                    #actualizo la cantidad de piezas verdes:
                    cantidad_piezas_verdes_anterior = cantidad_piezas_verdes_actual
                    tablero_piezas_verdes_anterior = tablero_verde
                    #print("El tablero actual de las piezas verdes es: ")
                    #print(tablero_piezas_verdes_anterior)
                    #print("-----------------------------------------------------------------------------------")             
                    print("asdasdaskkakaka")
        
        
                # El ser diferentes la cantidad de piezas, imlpica que una pieza fue comida
                elif cantidad_piezas_azules_anterior != cantidad_piezas_azules_actual:
            
            
                    # Detecto las nuevas posiciones de ambas piezas en conjunto (verdes y azules)
                    coordenadas_piezas = detectar_nuevas_posiciones(coordenadas_esquinas, frame)
                    tablero_actual = etiquetar_piezas_actuales(coordenadas_piezas, MatrizTraspuesta)    
        
                        # Obtengo la letra que mapeo su posicion
                    letra = letras(tablero_actual, tablero_inicial,turno)
                        #print("La letra detectada es: ", letra)
        
                        # Obtengo mi matriz con la jugada realizada
                    cambios, tablero_resultante = comparar_tableros(tablero_actual, tablero_inicial, letra)###

                
                
                        # De aca en adelante cambia al tener una pieza menos, osea agrego las funciones que me permiten obtener el tablero de cuando una pieza se come a la otra
            
                    tablero_modificado = actualizar_contadores(cantidad_piezas_azules_anterior, cantidad_piezas_azules_actual, tablero_piezas_azules_anterior, tablero_azul, tablero_resultante, letra)
                        #Imprimo el tablero modificado en caso de tener algo
                    #if tablero_modificado:
                    #    print("Tablero modificado:")
                    #    for fila in tablero_modificado:
                    #        print(' '.join(fila))
            
                    # VERIFICACION DE LA CORONACION DEL PEON BLANCO
                    tablero_resultante1 = coronar_peon(tablero_modificado, 'blanco')  # Coronar peones blancos
                
                    print("El tablero resultante essss:") #Tablero inical pero obtenido de otra matriz
                    for fila in tablero_resultante1 :
                        print(fila)   
            
            
                    #print("El tablero inicial essss:")
                    # Imprimo
                    #for fila in tablero_inicial:
                    #    print(fila)              
            
            
                    #print("El tablero modificado essss:") #Tablero inical pero obtenido de otra matriz
                    #for fila in tablero_modificado :
                    #    print(fila)                     
                            
                    # VALIDACION DE MOVIMIENTOS 
                    tablero_inicial1 = copy.deepcopy(tablero_inicial)
                    tablero_modificado1 = copy.deepcopy(tablero_resultante1)
                    tablero_modificado_sin_negras = eliminar_piezas_negras(tablero_modificado1)
                    tablero_inicial_sin_negras = eliminar_piezas_negras(tablero_inicial1)
                
                    mensaje = Validacion_de_piezas_blancas_al_comer(tablero_modificado_sin_negras ,tablero_inicial_sin_negras,tablero_inicial)######
                    #display_board(tablero_validado)                  

                    #print("El tablero modificado essss:")
                    # Imprimo
                    #for fila in tablero_modificado:
                    #    print(fila)              
            
                    cambios1 = comparar_tableros_blancas(tablero_resultante1, tablero_inicial)
                    print("Cambios detectados:")
                    print(cambios)
                    cambios2 = comparar_tableros2_blancas(tablero_resultante1, tablero_inicial)
                    print("Cambios detectados:")
                    print(cambios2)
                    cambios3 = Jugada_Blancas_Visualizar(cambios1, cambios2)
                    print(cambios3)
                
                    mensajes = cambios3
                    # Genero una visualizacion mas realista de las jugadas
                    tablero = generar_tablero(tablero_resultante1)
                    mostrar_tablero(tablero)
                    otro_mensaje = mensaje
                    TURNOS = "PIEZAS BLANCAS"
                    mostrar_tableroasd(tablero_resultante1, piece_images, mensajes, otro_mensaje, TURNOS)
                    mensajes = ""
                
                    #STOCKFISH:
                    print("las Piezas negras responden con: ")
                    # tablero_inicial 
                    board = jugar_como_negras(tablero_resultante1)
                    matriz_actualizada = jugada_Stockfish(board)
                    
                    #GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD
                    # ESPERO 3 SEGUNDOS ANTES DE MOSTRAR LA JUGADA DE STOCKFISH pero sin detener la camara
                    # Guardo el tiempo de inicio del delay
                    start_time = time.time()
        
                    # Esperar 3 segundos antes de mostrar el siguiente tablero
                    while time.time() - start_time < 3:
                        #
                        # Mientras se espera, se puede realizar algún procesamiento ligero si es necesario
                        # por ejemplo, leer la cámara o manejar otros eventos
                        ret, frame = cap.read()
                        frame = cv2.bitwise_and(frame, mascara)
                        cv2.imshow('PARTIDA', frame)
                        cv2.waitKey(1)              
                
                    cambios1 = comparar_tableros_negras(matriz_actualizada, tablero_resultante1)
                    print("Cambios detectados:")
                    print(cambios1)
                    cambios2 = comparar_tableros2_negras(matriz_actualizada, tablero_resultante1)
                    print("Cambios detectados:")
                    print(cambios2)
                    cambios3 = Jugada_Negras_Visualizar(cambios1, cambios2)
                    print(cambios3)
                    #mensajes = ""
                    mensajes = cambios3               
                    otro_mensaje = "EL MOVIMIENTO ES VALIDO."
                    TURNOS = "PIEZAS NEGRAS"
                    # ACAAAAAAAAAAAAAAAAAAAAA ESTA EL PROBLEMA ?????
                    mostrar_tableroasd(matriz_actualizada, piece_images, mensajes, otro_mensaje, TURNOS)
                    mensajes = ""
                    #GDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGDGD 
                
                    # Parametros de la jugada para pasarlo a arduino
                    jugada = jugada_obtenida_de_Stockfish(tablero_resultante, matriz_actualizada)
                    print("Stockfish jugo con: ",jugada)
                

                    # Le agrego las ultimas dos variables ( 0) si solo avanza o ( 1) si ademas de avanzar come
                    resultado = verificar_come_o_avanza(tablero_inicial, matriz_actualizada, jugada)
                    print(resultado)  # Resultado
                    #verificar_ultimo_elemento(resultado)
                    enviar_coordenadas_a_arduino(resultado)
                    
                    # Genero una visualizacion mas realista de las jugadas
                    tablero = generar_tablero(matriz_actualizada)
                    mostrar_tablero(tablero)            
            
                    # Reinicio
                    jugada = []
                    # Actualizo el tablero inicial con el tablero resultante
                    tablero_inicial = tablero_resultante1


                
                    # ACA IRIA EL CODIGO     DEL BRAZO ROBOT
                
                    # Vacio la variable letra
                    letra = ' '
                
                    #print("-----------------------------------------------------------------------------------")  
                    #print("El tablero anterior de las piezas azules es: ")
                    #print(tablero_piezas_azules_anterior)  
                    #actualizo la cantidad de piezas azules:
                    cantidad_piezas_azules_anterior = cantidad_piezas_azules_actual
                    tablero_piezas_azules_anterior = tablero_azul
                    #print("El tablero actual de las piezas azules es: ")
                    #print(tablero_piezas_azules_anterior)
                    #print("-----------------------------------------------------------------------------------")  
                    #print("El tablero anterior de las piezas verdes es: ")
                    #print(tablero_piezas_verdes_anterior)
                    #actualizo la cantidad de piezas verdes:
                    cantidad_piezas_verdes_anterior = cantidad_piezas_verdes_actual
                    tablero_piezas_verdes_anterior = tablero_verde
                    #print("El tablero actual de las piezas verdes es: ")
                    #print(tablero_piezas_verdes_anterior)
                    #print("-----------------------------------------------------------------------------------")  
                
#########################################################################################################
            # Piezas Negras = Piezas Azules
            if jugador_turno == "Negras": 
        
        
                print("-----------------------------------------------------------------------------------")   
                print("----------------------------Turno de las piezas Negras-----------------------------")  
        
                coordenadas_azul = detector_por_color2(Az1, Az2, Az3, Az4, Az5, Az6, frame, nombre_variable = "coordenadas_piezas_azules")
                tablero_azul = etiquetar_piezas_actuales(coordenadas_azul, MatrizTraspuesta)  
                cantidad_piezas_azules_actual = contar_piezas(tablero_azul)
                #print("La cantidad de piezas azules actual en el tablero es:", cantidad_piezas_azules_actual)
            
                coordenadas_verde = detector_por_color2(Ve1, Ve2, Ve3, Ve4, Ve5, Ve6, frame, nombre_variable = "coordenadas_piezas_verdes")
                tablero_verde = etiquetar_piezas_actuales(coordenadas_verde, MatrizTraspuesta)     
                cantidad_piezas_verdes_actual = contar_piezas(tablero_verde)  
                #print("La cantidad de piezas verdes actual en el tablero es:", cantidad_piezas_verdes_actual) 
        
            
                # El ser iguales la cantidades de las piezas, implica que ninguna pieza fue comida
                if cantidad_piezas_verdes_anterior  == cantidad_piezas_verdes_actual:    
            
                    # Detecto las nuevas posiciones de las piezas
                    coordenadas_piezas = detectar_nuevas_posiciones(coordenadas_esquinas, frame)
                    tablero_actual = etiquetar_piezas_actuales(coordenadas_piezas, MatrizTraspuesta)    
            
                    # Obtengo la letra que mapeo su posicion
                    letra = letras(tablero_actual, tablero_inicial,turno)
                    #print("La letra detectada es: ", letra)
        
                    # Obtengo mi matriz con la jugada realizada
                    cambios, tablero_resultante = comparar_tableros(tablero_actual, tablero_inicial, letra)
                    print(cambios)

                    print("El tablero inicial essss:") #Tablero inical pero obtenido de otra matriz
                    for fila in tablero_inicial :
                        print(fila)      
                
                    print("El tablero resultante essss:") #Tablero inical pero obtenido de otra matriz
                    for fila in tablero_resultante :
                        print(fila)         
                
                    # CORONACION DE STOCKFISH
                    tablero_resultante1 = coronacion_piezas_negras(tablero_inicial, tablero_resultante)
                
                
                    # VERIFICACION DEL ENROQUE CORTO
                    tablero_resultante3, numero = verificar_enroque_corto1(tablero_inicial, tablero_resultante1)##
                    #print(numero)
                
                    # VERIFICACION DEL ENROQUE LARGO
                    if numero != 1:
                        tablero_resultante3, numero = verificar_enroque_largo1(tablero_inicial, tablero_resultante3)
                    #print(numero)
                      
                    
                    # Imprimo
                    #for fila in tablero_resultante:
                    #    print(fila)   
                
                    # Genero una visualizacion mas realista de las jugadas
                    tablero = generar_tablero(tablero_resultante3)
                    mostrar_tablero(tablero)
    
                    cambios1 = comparar_tableros_negras(tablero_resultante3, tablero_inicial)
                    print("Cambios detectados:")
                    print(cambios1)
                    cambios2 = comparar_tableros2_negras(tablero_resultante3, tablero_inicial)
                    print("Cambios detectados:")
                    print(cambios2)
                    cambios3 = Jugada_Negras_Visualizar(cambios1, cambios2)
                    print(cambios3)
                    mensajes = cambios3              
                    #mensajes.append("Negras: A8 A6")
                    otro_mensaje = "EL MOVIMIENTO ES VALIDO."
                    TURNOS = "PIEZAS BLANCAS"
                    mostrar_tableroasd(tablero_resultante3, piece_images, mensajes, otro_mensaje, TURNOS)
                    mensajes = ""
                
                    jugada = []     
                
                    # Reseteo "numero" al valor original
                    numero = 0
                    # Actualizo el tablero inicial con el tablero resultante
                    tablero_inicial = tablero_resultante3
                
                    # Vacio la variable letra
                    letra = ' '
            
                    #print("-----------------------------------------------------------------------------------")  
                    #print("El tablero anterior de las piezas azules es: ")
                    #print(tablero_piezas_azules_anterior)  
                    #actualizo la cantidad de piezas azules:
                    cantidad_piezas_azules_anterior = cantidad_piezas_azules_actual
                    tablero_piezas_azules_anterior = tablero_azul
                    #print("El tablero actual de las piezas azules es: ")
                    #print(tablero_piezas_azules_anterior)
                    #print("-----------------------------------------------------------------------------------")  
                    #print("El tablero anterior de las piezas verdes es: ")
                    #print(tablero_piezas_verdes_anterior)
                    #actualizo la cantidad de piezas verdes:
                    cantidad_piezas_verdes_anterior = cantidad_piezas_verdes_actual
                    tablero_piezas_verdes_anterior = tablero_verde
                    #print("El tablero actual de las piezas verdes es: ")
                    #print(tablero_piezas_verdes_anterior)
                    #print("-----------------------------------------------------------------------------------")  
            
                # El ser diferentes la cantidad de piezas, imlpica que una pieza fue comida            
                elif  cantidad_piezas_verdes_anterior  != cantidad_piezas_verdes_actual:

                    
                    # Detecto las nuevas posiciones de las piezas
                    coordenadas_piezas = detectar_nuevas_posiciones(coordenadas_esquinas, frame)
                    tablero_actual = etiquetar_piezas_actuales(coordenadas_piezas, MatrizTraspuesta)    
            
                    # Obtengo la letra que mapeo su posicion
                    letra = letras(tablero_actual, tablero_inicial,turno)
                    #print("La letra detectada es: ", letra)
        
                    # Obtengo mi matriz con la jugada realizada
                    cambios, tablero_resultante = comparar_tableros(tablero_actual, tablero_inicial, letra)
                    #print(cambios)

                    # Imprimo
                    #for fila in tablero_resultante:
                    #    print(fila)   
            
                    # De aca en adelante son las funciones de cuando una pieza se come a otra
                
                    tablero_modificado = actualizar_contadores(cantidad_piezas_verdes_anterior, cantidad_piezas_verdes_actual, tablero_piezas_verdes_anterior, tablero_verde, tablero_resultante, letra)
                
                    print("El tablero actual essss:") #Tablero inical pero obtenido de otra matriz
                    for fila in tablero_modificado :
                        print(fila)       
                
                    matriz_actual = coronacion_piezas_negras(tablero_inicial, tablero_modificado)
            
                    print("El tablero pasado por la coronacion essss:") #Tablero inical pero obtenido de otra matriz
                    for fila in matriz_actual:
                        print(fila)       
                
            
                    # Genero una visualizacion mas realista de las jugadas
                    tablero = generar_tablero(matriz_actual)
                    mostrar_tablero(tablero)
                
                
                    cambios1 = comparar_tableros_negras(matriz_actual, tablero_inicial)
                    print("Cambios detectados:")
                    print(cambios1)
                    cambios2 = comparar_tableros2_negras(matriz_actual, tablero_inicial)
                    print("Cambios detectados:")
                    print(cambios2)
                    cambios3 = Jugada_Negras_Visualizar(cambios1, cambios2)
                    print(cambios3)
                    mensajes = cambios3
                    otro_mensaje = "EL MOVIMIENTO ES VALIDO."
                    TURNOS = "PIEZAS BLANCAS"
                    mostrar_tableroasd(matriz_actual, piece_images, mensajes, otro_mensaje, TURNOS)
                    mensajes = ""
                
                    jugada = []          
            
                    # Actualizo el tablero inicial con el tablero resultante
                    tablero_inicial = matriz_actual

                
                    # Vacio la variable letra
                    letra = ' '
            
                    #print("-----------------------------------------------------------------------------------")  
                    #print("El tablero anterior de las piezas azules es: ")
                    #print(tablero_piezas_azules_anterior)  
                    #actualizo la cantidad de piezas azules:
                    cantidad_piezas_azules_anterior = cantidad_piezas_azules_actual
                    tablero_piezas_azules_anterior = tablero_azul
                    #print("El tablero actual de las piezas azules es: ")
                    #print(tablero_piezas_azules_anterior)
                    #print("-----------------------------------------------------------------------------------")  
                    #print("El tablero anterior de las piezas verdes es: ")
                    #print(tablero_piezas_verdes_anterior)
                    #actualizo la cantidad de piezas verdes:
                    cantidad_piezas_verdes_anterior = cantidad_piezas_verdes_actual
                    tablero_piezas_verdes_anterior = tablero_verde
                    #print("El tablero actual de las piezas verdes es: ")
                    #print(tablero_piezas_verdes_anterior)
                    #print("-----------------------------------------------------------------------------------")              
#########################################################################################################

            count += 1    
                        
        
        
    # Manejo de eventos de teclado
    key = cv2.waitKey(1)
    if key == 27:  # Tecla ESC para salir del bucle
        break

# Liberar la cámara y cerrar todas las ventanas
cap.release()
cv2.destroyAllWindows()




NameError: name 'arduino' is not defined

<span style="color:yellow; font-size:32px; font-family:'Times New Roman'">**Codigo de Calibracion de Colores**</span>

<!-- 

color: cambia el color del texto 
font-size: establece el tamaño de la letra en píxeles.
font-family:'Times New Roman' cambia el tipo de letra a "Times New Roman".
** se utiliza para hacer que el texto del título esté en negrita

-->

In [ ]:
import cv2
import numpy as np


# Inicializo mi contador
count = 0

# Inicializo la cámara
cap = cv2.VideoCapture(1)

proceso_realizado = False  # Variable para controlar si se ha realizado el proceso

while True:
    ret, frame = cap.read()

    # Muestro la imagen en una ventana
    cv2.imshow('Reconocer Tablero', frame)

    # Manejo de eventos
    key = cv2.waitKey(1)

    if key == ord(' '):
        # Capturo la imagen si se presiona la tecla de espacio por primera vez
        filename = f'captura_{count}.png'
        cv2.imwrite(filename, frame)
        print(f"Captura guardada como {filename}")
        if not proceso_realizado:

            # Marcar que el proceso ha sido realizado
            proceso_realizado = True

    elif key == 27:  # Tecla ESC para salir del bucle
        break

    # Salir del bucle si el proceso ha sido realizado
    if proceso_realizado:
        break

# Liberar la cámara y cerrar todas las ventanas
cap.release()
cv2.destroyAllWindows()


# Cargo la imagen
#image = cv2.imread(r'C:\Users\Vegan\PROYECTO BRAZO ROBOT AJEDRECISTA\captura_0.png')
image = cv2.imread(filename)

# Reducir las dimensiones de la imagen en un 10%
scale_percent = 90 
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)
image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)

# Vectores para almacenar los valores HSV calibrados
color_amarillo = []
color_azul = []
color_verde = []
color_azul_verde = []

# Función para ajustar los valores HSV
def nothing(x):
    pass

# Creo una ventana para ajustar los valores HSV
cv2.namedWindow('Adjust HSV')

# Creo controles deslizantes para ajustar los valores HSV
cv2.createTrackbar('Hue min', 'Adjust HSV', 0, 180, nothing)
cv2.createTrackbar('Sat min', 'Adjust HSV', 0, 255, nothing)
cv2.createTrackbar('Val min', 'Adjust HSV', 0, 255, nothing)
cv2.createTrackbar('Hue max', 'Adjust HSV', 0, 180, nothing)
cv2.createTrackbar('Sat max', 'Adjust HSV', 0, 255, nothing)
cv2.createTrackbar('Val max', 'Adjust HSV', 0, 255, nothing)

# Contador para controlar el número de muestras
sample_count = 0

while True:
    # Obtengo los valores actuales de los controles deslizantes
    h_min = cv2.getTrackbarPos('Hue min', 'Adjust HSV')
    s_min = cv2.getTrackbarPos('Sat min', 'Adjust HSV')
    v_min = cv2.getTrackbarPos('Val min', 'Adjust HSV')
    h_max = cv2.getTrackbarPos('Hue max', 'Adjust HSV')
    s_max = cv2.getTrackbarPos('Sat max', 'Adjust HSV')
    v_max = cv2.getTrackbarPos('Val max', 'Adjust HSV')

    # Defino el rango de colores en HSV
    lower_color = np.array([h_min, s_min, v_min])
    upper_color = np.array([h_max, s_max, v_max])

    # Convierto la imagen de BGR a HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Creo una máscara con el rango de colores
    mask = cv2.inRange(hsv_image, lower_color, upper_color)

    # Aplico la máscara a la imagen original
    result = cv2.bitwise_and(image, image, mask=mask)

    # Muestro la imagen resultante
    cv2.imshow('Adjusted Color Detection', result)

    # Guardo los valores calibrados cuando se presiona la tecla Enter
    key = cv2.waitKey(1) & 0xFF
    if key == ord(' '):  #
        if sample_count == 0:
            color_amarillo = [h_min, s_min, v_min, h_max, s_max, v_max]
            print("Valores HSV para color amarillo:", color_amarillo)
        elif sample_count == 1:
            color_azul = [h_min, s_min, v_min, h_max, s_max, v_max]
            print("Valores HSV para color azul:", color_azul)
        elif sample_count == 2:
            color_verde = [h_min, s_min, v_min, h_max, s_max, v_max]
            print("Valores HSV para color verde:", color_verde)
        elif sample_count == 3:
            color_azul_verde = [h_min, s_min, v_min, h_max, s_max, v_max]
            print("Valores HSV para color azul-verde:", color_azul_verde)
        sample_count += 1
        if sample_count == 4:
            break
    elif key == ord('q'):
        break

cv2.destroyAllWindows()



# Extraer elementos individuales y guardar en variables separadas
Am1, Am2, Am3, Am4, Am5, Am6 = color_amarillo
Az1, Az2, Az3, Az4, Az5, Az6 = color_azul
Ve1, Ve2, Ve3, Ve4, Ve5, Ve6 = color_verde
AV1, AV2, AV3, AV4, AV5, AV6 = color_azul_verde
